<a href="https://colab.research.google.com/github/MODA-NYC/Agency-Name-Project/blob/main/Agency_Name_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Overview
This project aims to create a standardized list of Agency Names* and publish this as a dataset on the NYC Open Data portal. The primary goal is to enhance data legibility and interoperability by providing official, consistently formatted agency names. This will provide a clear canonical source for how to format Agency Names, improving data quality and saving time when joining datasets on the Agency Name field.

This project is being developed by the Data Governance team in the Office of Data and Analytics.

*The word “Agency” is colloquially used to mean a government organization that includes a New York City Agency, a Mayoral Office, or a Commission.

Project Plan document: https://docs.google.com/document/d/1u9-sZXUWdand1yIRmmKGbq7D5RAgD2puWoYvbP06a4g/edit?usp=sharing

GitHub repository (final location of the code and documentation of this project): https://github.com/MODA-NYC/Agency-Name-Project

Import Pandas.
Import and mount Google Drive to the Colab environment for file access.

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Web Scraping for https://www.nyc.gov/nyc-resources/agencies.page

Import required libraries (requests for HTTP requests, BeautifulSoup from bs4 for HTML parsing, and pandas for data manipulation). Define two functions to process and scrape agency information from a given URL (https://www.nyc.gov/nyc-resources/agencies.page). The process_agency_info function extracts and processes agency names, URLs, and descriptions from HTML list tags. The scrape_agency_list function performs a web scrape to collect agency data, handling possible request exceptions and storing the data in a pandas DataFrame.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def process_agency_info(li_tag):
    a_tag = li_tag.find('a', class_='name')
    name = a_tag.text.strip() if a_tag else ''
    url = a_tag.get('href') if a_tag else ''
    description = li_tag.get('data-desc', '')

    # Preprocess name for unique identification
    name_processed = name.lower().strip()

    return {
        'Name': name_processed,
        'Name - NYC.gov Agency List': name,
        'URL': url,
        'Description': description
    }

def scrape_agency_list(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        agencies_info = []
        for li_tag in soup.select('.alpha-list li'):
            agencies_info.append(process_agency_info(li_tag))

        return agencies_info
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return []

# URL for the List of NYC agencies
url = 'https://www.nyc.gov/nyc-resources/agencies.page'
agencies_info = scrape_agency_list(url)

# Load the scraped data into a DataFrame
df_nyc_gov_agency_list = pd.DataFrame(agencies_info)

# Display the DataFrame
#print(df_nyc_gov_agency_list.head())

In [ ]:
df_nyc_gov_agency_list.shape

(160, 4)

In [ ]:
df_nyc_gov_agency_list.head()

,Name,Name - NYC.gov Agency List,URL,Description
0,"actuary, nyc office of the (nycoa)","Actuary, NYC Office of the (NYCOA)",http://www.nyc.gov/actuary,"The New York City Office of the Actuary (""NYCO..."
1,"administrative justice coordinator, nyc office...","Administrative Justice Coordinator, NYC Office...",http://www.nyc.gov/ajc,The Office of the Administrative Justice Coord...
2,"administrative tax appeals, office of","Administrative Tax Appeals, Office of",http://www.nyc.gov/oata,The Office of Administrative Tax Appeals was e...
3,"administrative trials and hearings, office of ...","Administrative Trials and Hearings, Office of ...",http://www.nyc.gov/oath,The NYC Office of Administrative Trials and H...
4,"aging, department for the (nyc aging)","Aging, Department for the (NYC Aging)",http://www.nyc.gov/aging,NYC Aging funds community-based organizations ...


In [ ]:
# Show duplicate entries based on the 'Name' column in df_nyc_gov_agency_list
duplicates_nyc_gov_agency_list = df_nyc_gov_agency_list[df_nyc_gov_agency_list.duplicated('Name', keep=False)]
duplicates_nyc_gov_agency_list

,Name,Name - NYC.gov Agency List,URL,Description


# Web Scrapping for https://www.nyc.gov/office-of-the-mayor/admin-officials.page

Define functions to scrape and process information from the https://www.nyc.gov/office-of-the-mayor/admin-officials.page. The process_mayor_office_info function extracts agency names, URLs, contact names, and titles from HTML elements and standardizes agency names for unique identification. The scrape_mayor_office_list function uses the requests library to fetch the webpage, parses it with BeautifulSoup, and aggregates the data into a list, handling exceptions gracefully. The results are loaded into a pandas DataFrame.


In [ ]:
def process_mayor_office_info(li_tag, source_name):
    agency_tag = li_tag.find('div', class_='al-agency').find('a')
    agency_name = agency_tag.text.strip() if agency_tag else ''
    agency_url = agency_tag.get('href') if agency_tag else ''

    contact_name_tag = li_tag.find('div', class_='al-contact').find('a')
    contact_name = contact_name_tag.text.strip() if contact_name_tag else ''
    contact_title_tag = li_tag.find('li', class_='al-contact-info')
    contact_title = contact_title_tag.text.strip() if contact_title_tag else ''

    # Modify the 'Name - NYC.gov Mayor's Office' field based on the specified condition
    if agency_name == "Mayor, Office of the":
        agency_name = "Office of the " + contact_title

    # Preprocess name for unique identification
    name_processed = agency_name.lower().strip()

    return {
        'Name': name_processed,
        'Name - NYC.gov Mayor\'s Office': agency_name,
        'URL': agency_url,
        'Contact Name': contact_name,
        'Contact Title': contact_title
    }

def scrape_mayor_office_list(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Ensure we raise an error for bad status
        soup = BeautifulSoup(response.text, 'html.parser')
        officials_info = []
        for li_tag in soup.select('li[data-topic]'):
            officials_info.append(process_mayor_office_info(li_tag, 'NYC.gov Mayor\'s Office'))
        return officials_info
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return []

# URL for the Office of the Mayor officials
url_mayor_office = 'https://www.nyc.gov/office-of-the-mayor/admin-officials.page'
officials_info = scrape_mayor_office_list(url_mayor_office)

# Load the scraped data into a DataFrame
df_nyc_mayor_office = pd.DataFrame(officials_info)

# Display the DataFrame
df_nyc_mayor_office.head()

,Name,Name - NYC.gov Mayor's Office,URL,Contact Name,Contact Title
0,"actuary, nyc office of the (nycoa)","Actuary, NYC Office of the (NYCOA)",http://www.nyc.gov/actuary,Marek Tyszkiewicz,Chief Actuary
1,"administrative justice coordinator, nyc office...","Administrative Justice Coordinator, NYC Office...",http://www.nyc.gov/ajc,David Goldin,Administrative Justice Coordinator
2,"administrative tax appeals, office of","Administrative Tax Appeals, Office of",http://www.nyc.gov/oata,Frances Henn,Director
3,"administrative trials and hearings, office of ...","Administrative Trials and Hearings, Office of ...",http://www.nyc.gov/oath,Asim Rehman,Commissioner
4,"aging, department for the (nyc aging)","Aging, Department for the (NYC Aging)",http://www.nyc.gov/aging,Lorraine A. Cortés-Vázquez,Commissioner


In [ ]:
# Show duplicate entries based on the 'Name' column in df_nyc_mayor_office
duplicates_nyc_mayor_office = df_nyc_mayor_office[df_nyc_mayor_office.duplicated('Name', keep=False)]
duplicates_nyc_mayor_office

,Name,Name - NYC.gov Mayor's Office,URL,Contact Name,Contact Title


# Web Scrapper for https://opendata.cityofnewyork.us/data/

Define a function to scrape agency information from the NYC Open Data Portal. The scrape_open_data_list function fetches the page using requests, parses it with BeautifulSoup, and iterates through specified HTML elements to collect agency names and URLs. Agency names are processed for uniformity. The collected data is stored in a pandas DataFrame for further manipulation and analysis.

In [ ]:
def scrape_open_data_list(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    data_info = []

    for ul_tag in soup.select('div.content-block ul.space-section'):
        for li_tag in ul_tag.select('li'):
            a_tag = li_tag.find('a')
            if a_tag:
                agency_name = a_tag.text.strip()
                agency_url = a_tag.get('href')

                # Preprocess name for unique identification
                name_processed = agency_name.lower().strip()

                data_info.append({
                    'Name': name_processed,
                    'Name - NYC Open Data Portal': agency_name,
                    'URL': agency_url
                })

    return data_info

# URL for the NYC Open Data Portal
url_open_data = 'https://opendata.cityofnewyork.us/data/'
open_data_info = scrape_open_data_list(url_open_data)

# Load the scraped data into a DataFrame
df_nyc_open_data_portal = pd.DataFrame(open_data_info)

# Display the DataFrame
#print(df_nyc_open_data_portal.head())

In [ ]:
df_nyc_open_data_portal.head()

,Name,Name - NYC Open Data Portal,URL
0,administration for children’s services (acs),Administration for Children’s Services (ACS),https://data.cityofnewyork.us/browse?Dataset-I...
1,board of elections (boeny),Board of Elections (BOENY),https://data.cityofnewyork.us/browse?Dataset-I...
2,board of standards and appeals (bsa),Board of Standards and Appeals (BSA),https://data.cityofnewyork.us/browse?Dataset-I...
3,bronx borough president (bpbx),Bronx Borough President (BPBX),https://data.cityofnewyork.us/browse?Dataset-I...
4,brooklyn borough president (bpbk),Brooklyn Borough President (BPBK),https://data.cityofnewyork.us/browse?Dataset-I...


In [ ]:
# Show duplicate entries based on the 'Name' column in df_nyc_open_data_portal
duplicates_nyc_open_data_portal = df_nyc_open_data_portal[df_nyc_open_data_portal.duplicated('Name', keep=False)]
duplicates_nyc_open_data_portal


,Name,Name - NYC Open Data Portal,URL


# Web Scrapper for https://www.checkbooknyc.com/agency_codes/newwindow

Checkbook data

In [ ]:
def load_checkbook_data():
    checkbook_url = 'https://www.checkbooknyc.com/agency_codes/newwindow'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(checkbook_url, headers=headers)

    if response.status_code != 200:
        print("Failed to retrieve the webpage")
        return pd.DataFrame()

    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the table containing the data
    table = soup.find('table')
    if table is None:
        print("No table found on the webpage")
        return pd.DataFrame()

    # Extract table rows
    rows = table.find_all('tr')

    data = []
    for row in rows:
        cols = row.find_all('td')
        if len(cols) > 2:
            code = cols[0].text.strip()
            name = cols[1].text.strip()  # Keep the original name formatting
            short_name = cols[2].text.strip()  # Extract the "Agency Short Name"
            data.append({
                'Code': code,
                'Name': name.lower(),  # Preprocess the name to lowercase for merging
                'Name - Checkbook': name,
                'Agency Short Name': short_name
            })

    df_checkbook = pd.DataFrame(data)

    return df_checkbook[['Name', 'Name - Checkbook', 'Agency Short Name']]

# Load and process Checkbook NYC data
df_checkbook = load_checkbook_data()

In [ ]:
df_checkbook.head()

,Name,Name - Checkbook,Agency Short Name
0,administration for children's services,Administration for Children's Services,ADM CHILD SV
1,board of correction,Board of Correction,BD CORRECTN
2,board of elections,Board of Elections,BD ELECTIONS
3,borough president - bronx,Borough President - Bronx,BP BRONX
4,borough president - brooklyn,Borough President - Brooklyn,BP BROOKLYN


In [ ]:
# Show duplicate entries based on the 'Name' column in df_checkbook
duplicates_checkbook = df_checkbook[df_checkbook.duplicated('Name', keep=False)]
duplicates_checkbook

,Name,Name - Checkbook,Agency Short Name


# Load Greenbook data from the Open Data Portal and Preprocess the data

In [ ]:
import requests

# URL of the data endpoint
data_url = 'https://data.cityofnewyork.us/resource/mdcw-n682.json'

# Initialize a list to store dataframe chunks
dataframes = []
offset = 0
limit = 1000  # Adjust if necessary based on the API limits

while True:
    # Append the offset and limit parameters to the query
    response = requests.get(f"{data_url}?$limit={limit}&$offset={offset}")
    data_chunk = pd.read_json(response.text)

    # If no data is returned, we've read all rows, so break the loop
    if data_chunk.empty:
        break

    # Append the chunk to the list of dataframes
    dataframes.append(data_chunk)

    # Increase the offset to get the next chunk of data
    offset += limit

# Concatenate all chunks into a single DataFrame
df_greenbook = pd.concat(dataframes, ignore_index=True)

# Copy the original name field to a new column with the dataset specific name
df_greenbook['Name - Greenbook'] = df_greenbook['agency_name'].copy()

# Preprocess the "Name" field to trim whitespace and convert to lowercase for unique identification
df_greenbook['Name'] = df_greenbook['agency_name'].str.lower().str.strip()

# Select only the 'Name' and 'Name - Greenbook' columns
df_greenbook = df_greenbook[['Name', 'Name - Greenbook']]

# Drop duplicate rows based on the 'Name' column
df_greenbook = df_greenbook.drop_duplicates(subset=['Name'])

In [ ]:
df_greenbook.head()

,Name,Name - Greenbook
0,"actuary, office of","Actuary, Office of"
7,"administrative trials and hearings, office of","Administrative Trials And Hearings, Office of"
26,"health & mental hygiene, department of","Health & Mental Hygiene, Department of"
27,"aging, department for the","Aging, Department for the"
48,borough historians,Borough Historians


In [ ]:
# Show duplicate entries based on the 'Name' column in df_greenbook
duplicates_greenbook = df_greenbook[df_greenbook.duplicated('Name', keep=False)]
duplicates_greenbook

,Name,Name - Greenbook



# Load and Preprocess ODA Agency Data from CSV:

Loads a CSV file located on Google Drive into a DataFrame. The original 'Name' field is duplicated into a new column labeled 'Name - ODA'. The 'Name' field is then processed to remove whitespace and convert to lowercase for uniformity and ease of comparison or linking with other datasets.

In [ ]:
# Path to the CSV file
file_path = '/content/drive/MyDrive/Projects/ODA/Agency Name Project/ODA Data.csv'

# Load the CSV file into a DataFrame
df_oda_data = pd.read_csv(file_path)

# Copy the original name field to a new column with the dataset specific name
df_oda_data['Name - ODA'] = df_oda_data['Name'].copy()

# Preprocess the "Name" field to trim whitespace and convert to lowercase for unique identification
df_oda_data['Name'] = df_oda_data['Name'].str.lower().str.strip()

# Display the DataFrame
#print(df_oda_data.head())

In [ ]:
df_oda_data.head()

,Name,Agency Code,Parent Organization,Child Organization(s),Acronym,Agency Type,Website,Name - ODA
0,administration for children's services (acs),068,NaN,NaN,ACS,City Department,https://www.nyc.gov/site/acs/index.page,Administration for Children's Services (ACS)
1,association for a better new york (abny),NaN,NaN,NaN,ABNY,Other,abny.org,Association for a Better New York (ABNY)
2,board of correction (boc),073,NaN,NaN,BOC,Other,https://www.nyc.gov/site/boc/index.page,Board of Correction (BOC)
3,board of education retirement system (bers),NaN,NaN,NaN,BERS,Other,https://www.bers.nyc.gov/,Board of Education Retirement System (BERS)
4,board of elections (boe),003,NaN,NaN,BOE,Other,vote.nyc,Board of Elections (BOE)


In [ ]:
# Show duplicate entries based on the 'Name' column in df_oda_data
duplicates_oda_data = df_oda_data[df_oda_data.duplicated('Name', keep=False)]
duplicates_oda_data


,Name,Agency Code,Parent Organization,Child Organization(s),Acronym,Agency Type,Website,Name - ODA



# Load and Process Chief Privacy Officer (CPO) Data:

Loads a specific CSV file containing agency data from the Chief Privacy Officer (CPO) into a DataFrame. Renames and preprocesses the 'Agency or Office' column for consistent identification across datasets by trimming whitespace and converting to lowercase. The original column name is retained under 'Name - CPO' for reference. Finally, the original 'Agency or Office' column is dropped to streamline the DataFrame.

In [ ]:
import pandas as pd

# Path to the CSV file
cpo_file_path = '/content/drive/MyDrive/Projects/ODA/Agency Name Project/CPO Data.csv'

# Load the CSV file into a DataFrame
df_cpo_data = pd.read_csv(cpo_file_path)

# Assume 'Agency or Office' is the column we want to rename and preprocess
# Copy the original 'Agency or Office' to 'Name - CPO' before preprocessing
df_cpo_data['Name - CPO'] = df_cpo_data['Agency or Office'].copy()

# Preprocess 'Agency or Office' for unique identification (trim and lower case)
df_cpo_data['Name'] = df_cpo_data['Agency or Office'].str.lower().str.strip()

# Now we can drop the original 'Agency or Office' column if it's no longer needed
df_cpo_data.drop(columns=['Agency or Office'], inplace=True)

# Display the DataFrame
#print(df_cpo_data.head())

In [ ]:
df_cpo_data.head()

,Acronym,Name - CPO,Name
0,ACS,Administration for Children's Services,administration for children's services
1,BOC,Board of Correction,board of correction
2,BERS,Board of Education Retirement System,board of education retirement system
3,BSA,Board of Standards and Appeals,board of standards and appeals
4,Bronx BP,Bronx Borough President's Office,bronx borough president's office


In [ ]:
# Show duplicate entries based on the 'Name' column in df_cpo_data
duplicates_cpo_data = df_cpo_data[df_cpo_data.duplicated('Name', keep=False)]
duplicates_cpo_data


,Acronym,Name - CPO,Name


# Load, Process, and Filter WeGov Data

Loads a CSV file containing data from the civic group WeGov into a DataFrame. Copies and renames the 'name' column to 'Name - WeGov', then preprocesses the 'name' for uniformity by trimming and converting to lowercase. The original 'name' column is removed post-processing. Additionally, filters the dataset to include only rows where the 'type' column values are 'City Agency' or 'Elected Office', focusing on relevant entities for further analysis.

In [ ]:
# Path to the CSV file
wegov_file_path = '/content/drive/MyDrive/Projects/ODA/Agency Name Project/WeGov Data.csv'

# Load the CSV file into a DataFrame
df_wegov_data = pd.read_csv(wegov_file_path)

# Copy the original 'name' to 'Name - WeGov' before renaming
df_wegov_data['Name - WeGov'] = df_wegov_data['name'].copy()

# Preprocess 'name' for unique identification (trim and lower case)
df_wegov_data['Name'] = df_wegov_data['name'].str.lower().str.strip()

# Drop the original 'name' column as its data has been preserved and preprocessed
df_wegov_data.drop(columns=['name'], inplace=True)

# Filter the DataFrame for rows where the "type" column is either "City Agency" or "Elected Office"
#df_wegov_data['type'] = df_wegov_data['type'].str.strip()
df_wegov_data = df_wegov_data[df_wegov_data['type'].isin(['City Agency', 'Elected Office'])]

# Display the DataFrame
#print(filtered_df_wegov_data.head())

In [ ]:
df_wegov_data.head()

,type,Name - WeGov,Name
4,City Agency,NYC Municipal Water Finance Authority,nyc municipal water finance authority
5,City Agency,NYC Technology Development Corporation,nyc technology development corporation
6,City Agency,Office of Administrative Tax Appeals,office of administrative tax appeals
7,City Agency,Transitional Finance Authority,transitional finance authority
24,Elected Office,Mayor's Office,mayor's office


In [ ]:
# Show duplicate entries based on the 'Name' column in df_wegov_data
duplicates_wegov_data = df_wegov_data[df_wegov_data.duplicated('Name', keep=False)]
duplicates_wegov_data

,type,Name - WeGov,Name


# Standardize and Normalize Agency Names Across DataFrames

Imports re for regular expressions and unicodedata for character normalization. Defines a function standardize_name to normalize, clean, and reformat agency names for consistency. This includes decomposing characters, replacing special characters, removing acronyms, and adjusting name order. Applies this standardized function to the 'Name' column of each DataFrame containing agency information, ensuring uniform naming across multiple data sources. This process aids in data integration and comparison.

In [ ]:
import re
import unicodedata

def standardize_name(name):
    # Normalize the string to decompose combined characters and replace special characters
    name = unicodedata.normalize('NFKD', name)
    name = name.replace('’', "'").replace('‘', "'")

    # Remove extra spaces and invisible characters
    name = "".join(char for char in name if unicodedata.category(char).strip())

    # Expand common abbreviations
    name = re.sub(r'\bdept\b', 'department', name, flags=re.IGNORECASE)
    name = re.sub(r'\b&\b', 'and', name)

    # Convert to ASCII
    name = name.encode('ascii', 'ignore').decode('ascii')

    # Remove punctuation except parentheses
    name = re.sub(r'[^\w\s\(\)]', '', name)

    # Remove multiple spaces
    name = re.sub(r'\s+', ' ', name)

    return name.lower().strip()

# Re-apply the standardized function to each dataframe's 'Name' field with acronyms included
dataframes = [df_nyc_gov_agency_list, df_nyc_mayor_office, df_nyc_open_data_portal, df_oda_data, df_cpo_data, df_wegov_data, df_greenbook, df_checkbook]

for df in dataframes:
    df['Name'] = df['Name'].apply(standardize_name)

# Combine dataframes into one for further processing
combined_df = pd.concat(dataframes, ignore_index=True)

# Display the first few rows to verify
#print(combined_df.head())

In [ ]:
combined_df.head()

,Name,Name - NYC.gov Agency List,URL,Description,Name - NYC.gov Mayor's Office,Contact Name,Contact Title,Name - NYC Open Data Portal,Agency Code,Parent Organization,...,Acronym,Agency Type,Website,Name - ODA,Name - CPO,type,Name - WeGov,Name - Greenbook,Name - Checkbook,Agency Short Name
0,actuary nyc office of the (nycoa),"Actuary, NYC Office of the (NYCOA)",http://www.nyc.gov/actuary,"The New York City Office of the Actuary (""NYCO...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,administrative justice coordinator nyc office ...,"Administrative Justice Coordinator, NYC Office...",http://www.nyc.gov/ajc,The Office of the Administrative Justice Coord...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,administrative tax appeals office of,"Administrative Tax Appeals, Office of",http://www.nyc.gov/oata,The Office of Administrative Tax Appeals was e...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,administrative trials and hearings office of (...,"Administrative Trials and Hearings, Office of ...",http://www.nyc.gov/oath,The NYC Office of Administrative Trials and H...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,aging department for the (nyc aging),"Aging, Department for the (NYC Aging)",http://www.nyc.gov/aging,NYC Aging funds community-based organizations ...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Display DataFrame Names and Sizes

In [ ]:
dataframes = {
    'NYC Gov Agency List': df_nyc_gov_agency_list,
    'NYC Mayor Office': df_nyc_mayor_office,
    'NYC Open Data Portal': df_nyc_open_data_portal,
    'ODA Data': df_oda_data,
    'CPO Data': df_cpo_data,
    'WeGov Data': df_wegov_data,
    'Greenbook Data': df_greenbook,
    'Checkbook Data': df_checkbook
}

# Print the name and shape of each dataframe
for name, df in dataframes.items():
    print(f"{name}: {df.shape}")

NYC Gov Agency List: (160, 4)
NYC Mayor Office: (177, 5)
NYC Open Data Portal: (89, 3)
ODA Data: (183, 8)
CPO Data: (186, 3)
WeGov Data: (180, 3)
Greenbook Data: (124, 2)
Checkbook Data: (145, 3)


# Combine DataFrames with Agency Names from Multiple Sources

Initializes a combined DataFrame using the 'Name' and 'Name - NYC.gov Agency List' columns from the NYC government agency list. Constructs a list of tuples, each containing a DataFrame and its respective unique agency name column. Iterates through this list, merging each DataFrame with the combined DataFrame based on the standardized 'Name' field, using an outer join to ensure all data is included. The result is a comprehensive DataFrame that aligns agency names across different sources, useful for data comparison and integration.

In [ ]:
# Initialize the combined dataframe with the first dataframe's relevant columns
combined_df = df_nyc_gov_agency_list[['Name', 'Name - NYC.gov Agency List']]

# List of tuples containing dataframes and their respective "Name - Dataset" columns
dataframes_to_merge = [
    (df_nyc_mayor_office, 'Name - NYC.gov Mayor\'s Office'),
    (df_nyc_open_data_portal, 'Name - NYC Open Data Portal'),
    (df_oda_data, 'Name - ODA'),
    (df_cpo_data, 'Name - CPO'),
    (df_wegov_data, 'Name - WeGov'),
    (df_greenbook, 'Name - Greenbook'),
    (df_checkbook, 'Name - Checkbook')
]

# Merge each dataframe in the list with the combined dataframe
for df, name_column in dataframes_to_merge:
    combined_df = combined_df.merge(df[['Name', name_column]], on='Name', how='outer')

# Display the head of the combined dataframe to verify
#print(combined_df.head())

In [ ]:
combined_df.shape

(763, 9)

In [ ]:
combined_df.head()

,Name,Name - NYC.gov Agency List,Name - NYC.gov Mayor's Office,Name - NYC Open Data Portal,Name - ODA,Name - CPO,Name - WeGov,Name - Greenbook,Name - Checkbook
0,actuary nyc office of the (nycoa),"Actuary, NYC Office of the (NYCOA)","Actuary, NYC Office of the (NYCOA)",NaN,NaN,NaN,NaN,NaN,NaN
1,administrative justice coordinator nyc office ...,"Administrative Justice Coordinator, NYC Office...","Administrative Justice Coordinator, NYC Office...",NaN,NaN,NaN,NaN,NaN,NaN
2,administrative tax appeals office of,"Administrative Tax Appeals, Office of","Administrative Tax Appeals, Office of",NaN,NaN,NaN,NaN,NaN,NaN
3,administrative trials and hearings office of (...,"Administrative Trials and Hearings, Office of ...","Administrative Trials and Hearings, Office of ...",NaN,NaN,NaN,NaN,NaN,NaN
4,aging department for the (nyc aging),"Aging, Department for the (NYC Aging)","Aging, Department for the (NYC Aging)",NaN,NaN,NaN,NaN,NaN,NaN


# Extract Acronym and Remove it from the Name Field

In [ ]:
import re

def extract_acronym(name):
    # Extract acronym from parentheses
    match = re.search(r'\((.*?)\)', name)
    return match.group(1).upper() if match else ''

# Apply the function to create the Acronym field
combined_df['Acronym'] = combined_df['Name'].apply(extract_acronym)

# Display the updated DataFrame
print(combined_df[['Name', 'Acronym']].head())

                                                Name    Acronym
0                  actuary nyc office of the (nycoa)      NYCOA
1  administrative justice coordinator nyc office ...        AJC
2               administrative tax appeals office of           
3  administrative trials and hearings office of (...       OATH
4               aging department for the (nyc aging)  NYC AGING


In [ ]:
def remove_acronym(name):
    # Remove the acronym if present
    return re.sub(r'\s*\([^)]+\)\s*', '', name).strip()

# Apply the function to create a new Name field without acronyms
combined_df['Name'] = combined_df['Name'].apply(remove_acronym)

# Display the updated DataFrame
print(combined_df[['Name', 'Acronym']].head())

                                               Name    Acronym
0                         actuary nyc office of the      NYCOA
1  administrative justice coordinator nyc office of        AJC
2              administrative tax appeals office of           
3      administrative trials and hearings office of       OATH
4                          aging department for the  NYC AGING


In [ ]:
combined_df.shape

(763, 10)

# Merge just based on Acronym

In [ ]:
# Function to merge records based on matching acronyms
def merge_on_acronym(df):
    # Create a dictionary to hold merged records
    merged_records = {}
    no_acronym_records = []

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        acronym = row['Acronym']
        if acronym:
            if acronym in merged_records:
                # Merge the current record with the existing record
                for col in df.columns:
                    if pd.notnull(row[col]):
                        if col not in merged_records[acronym]:
                            merged_records[acronym][col] = row[col]
                        elif pd.isnull(merged_records[acronym][col]):
                            merged_records[acronym][col] = row[col]
                        elif col.startswith('Name -') and row[col] not in merged_records[acronym][col]:
                            merged_records[acronym][col] += '; ' + row[col]
            else:
                # Add the record to the dictionary
                merged_records[acronym] = row.to_dict()
        else:
            # Add records without acronyms to a separate list
            no_acronym_records.append(row.to_dict())

    # Convert the merged records dictionary back to a DataFrame
    merged_df = pd.DataFrame.from_dict(merged_records, orient='index')

    # Convert the no_acronym_records list back to a DataFrame
    no_acronym_df = pd.DataFrame(no_acronym_records)

    # Concatenate the merged_df and no_acronym_df DataFrames
    final_df = pd.concat([merged_df, no_acronym_df], ignore_index=True)

    return final_df

# Perform the merge based on acronym
merged_df = merge_on_acronym(combined_df)

# Display the merged DataFrame
print(merged_df.head())

# Save the merged DataFrame to a CSV file if needed
# merged_df.to_csv('merged_agency_names.csv', index=False)

# Continue with the dedupe process using merged_df
combined_df = merged_df

                                               Name  \
0                         actuary nyc office of the   
1  administrative justice coordinator nyc office of   
2      administrative trials and hearings office of   
3                          aging department for the   
4                     appointments mayors office of   

                          Name - NYC.gov Agency List  \
0                 Actuary, NYC Office of the (NYCOA)   
1  Administrative Justice Coordinator, NYC Office...   
2  Administrative Trials and Hearings, Office of ...   
3              Aging, Department for the (NYC Aging)   
4              Appointments, Mayor's Office of (MOA)   

                       Name - NYC.gov Mayor's Office  \
0                 Actuary, NYC Office of the (NYCOA)   
1  Administrative Justice Coordinator, NYC Office...   
2  Administrative Trials and Hearings, Office of ...   
3              Aging, Department for the (NYC Aging)   
4              Appointments, Mayor's Office of (MOA)

In [ ]:
combined_df.shape

(693, 10)

# Merge just based on Name field

In [ ]:
# Function to merge records based on matching names
def merge_on_name(df):
    # Create a dictionary to hold merged records
    merged_records = {}
    no_name_records = []

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        name = row['Name']
        if name:
            if name in merged_records:
                # Merge the current record with the existing record
                for col in df.columns:
                    if pd.notnull(row[col]):
                        if col not in merged_records[name]:
                            merged_records[name][col] = row[col]
                        elif pd.isnull(merged_records[name][col]):
                            merged_records[name][col] = row[col]
                        elif col.startswith('Acronym -') and row[col] not in merged_records[name][col]:
                            merged_records[name][col] += '; ' + row[col]
            else:
                # Add the record to the dictionary
                merged_records[name] = row.to_dict()
        else:
            # Add records without names to a separate list
            no_name_records.append(row.to_dict())

    # Convert the merged records dictionary back to a DataFrame
    merged_df = pd.DataFrame.from_dict(merged_records, orient='index')

    # Convert the no_name_records list back to a DataFrame
    no_name_df = pd.DataFrame(no_name_records)

    # Concatenate the merged_df and no_name_df DataFrames
    final_df = pd.concat([merged_df, no_name_df], ignore_index=True)

    return final_df

# Perform the merge based on acronym
merged_df = merge_on_acronym(combined_df)

# Perform the merge based on name
merged_df = merge_on_name(merged_df)

# Display the merged DataFrame
print(merged_df.head())

# Save the merged DataFrame to a CSV file if needed
# merged_df.to_csv('merged_agency_names.csv', index=False)

# Continue with the dedupe process using merged_df
combined_df = merged_df

                                               Name  \
0                         actuary nyc office of the   
1  administrative justice coordinator nyc office of   
2      administrative trials and hearings office of   
3                          aging department for the   
4                     appointments mayors office of   

                          Name - NYC.gov Agency List  \
0                 Actuary, NYC Office of the (NYCOA)   
1  Administrative Justice Coordinator, NYC Office...   
2  Administrative Trials and Hearings, Office of ...   
3              Aging, Department for the (NYC Aging)   
4              Appointments, Mayor's Office of (MOA)   

                       Name - NYC.gov Mayor's Office  \
0                 Actuary, NYC Office of the (NYCOA)   
1  Administrative Justice Coordinator, NYC Office...   
2  Administrative Trials and Hearings, Office of ...   
3              Aging, Department for the (NYC Aging)   
4              Appointments, Mayor's Office of (MOA)

In [ ]:
combined_df.shape

(585, 10)

# Merging to dedupe - use generated list of confirmed matches to deduplicate the dataframe

I worked directly in several ChatGPT instances to develop and execute code that generated potential match lists. Then I manually reviewed the output and labeled all of the potential pairs. I then deduped the combined_df and performed a second round of generating matched pairs and labeled those. I then combined the matched pairs as consolidated_matches

In [ ]:
import pandas as pd

# Load the data
consolidated_matches_path = '/content/drive/MyDrive/Projects/ODA/Agency Name Project/Output/consolidated_matches.csv'
consolidated_matches = pd.read_csv(consolidated_matches_path)

# Assuming combined_data exists in your notebook as a dataframe called combined_df
# combined_df = ...

# Normalize Names
consolidated_matches['Name_1'] = consolidated_matches['Name_1'].str.lower().str.strip()
consolidated_matches['Name_2'] = consolidated_matches['Name_2'].str.lower().str.strip()
combined_df['Name'] = combined_df['Name'].str.lower().str.strip()

# Filter the matches to only include "Match" records
filtered_matches = consolidated_matches[consolidated_matches['Label'] == 'Match']

# Create a new mapping dictionary from filtered matches
name_mapping_filtered = pd.Series(filtered_matches['Name_2'].values,
                                  index=filtered_matches['Name_1']).to_dict()

# Function to remove cycles from the mapping
def remove_cycles(mapping):
    resolved_mapping = {}
    visited = set()

    def resolve(name):
        path = []
        while name in mapping and name not in visited:
            path.append(name)
            visited.add(name)
            name = mapping[name]
        for p in path:
            resolved_mapping[p] = name
        return name

    for key in mapping:
        resolve(key)

    return resolved_mapping

# Remove cycles from the mapping
resolved_mapping = remove_cycles(name_mapping_filtered)

# Function to map variations to standardized names
def map_name(name, mapping):
    return mapping.get(name, name)

# Apply the Mapping directly to the 'Name' field
combined_df['Name'] = combined_df['Name'].apply(lambda name: map_name(name, resolved_mapping))

# Reapply mapping to ensure consistency
combined_df['Name'] = combined_df['Name'].apply(lambda name: map_name(name, resolved_mapping))

# Create a new column "Merged Names" to store the list of merged names
merged_names_dict = combined_df.groupby('Name')['Name'].apply(lambda x: '; '.join(x.unique())).to_dict()
combined_df['Merged Names'] = combined_df['Name'].map(merged_names_dict)

# Aggregate Data, keeping the first non-null value for each column
deduplicated_df = combined_df.groupby('Name').first().reset_index()

# Ensure the "Merged Names" column is properly aggregated
deduplicated_df['Merged Names'] = deduplicated_df['Name'].map(merged_names_dict)

# Display the deduplicated dataframe
deduplicated_df.head()

,Name,Name - NYC.gov Agency List,Name - NYC.gov Mayor's Office,Name - NYC Open Data Portal,Name - ODA,Name - CPO,Name - WeGov,Name - Greenbook,Name - Checkbook,Acronym,Merged Names
0,actuary office of,"Actuary, NYC Office of the (NYCOA)","Actuary, NYC Office of the (NYCOA)",None,Office of the Actuary,Office of the Actuary,Office of the Actuary,"Actuary, Office of",Office of the Actuary,NYCOA,actuary office of
1,administration for childrens services,"Children's Services, Administration for (ACS)","Children's Services, Administration for (ACS)",Administration for Children’s Services (ACS),Administration for Children's Services (ACS),Administration for Children's Services,Administration for Children's Services,"Children's Services, Administration for",Administration for Children's Services,ACS,administration for childrens services
2,association for a better new york,None,None,None,Association for a Better New York (ABNY),None,None,None,None,ABNY,association for a better new york
3,board of correction,"Correction, Board of (BOC)","Correction, Board of (BOC)",None,Board of Correction (BOC),Board of Correction,Board of Correction,"Correction, Board of",Board of Correction,BOC,board of correction
4,board of elections,"Elections, Board of (BOE)","Elections, Board of (BOE)",Board of Elections (BOENY),Board of Elections (BOE),None,Board of Elections,"Elections, Board of",Board of Elections,BOE,board of elections


In [ ]:
deduplicated_df.shape

(337, 11)

In [ ]:
# Order the dataframe by 'Name' in ascending order
deduplicated_df = deduplicated_df.sort_values(by='Name').reset_index(drop=True)

# Add an 'ID' column with sequential numbers for each row
deduplicated_df['ID'] = range(1, len(deduplicated_df) + 1)

# Display the updated dataframe
deduplicated_df.head()

,Name,Name - NYC.gov Agency List,Name - NYC.gov Mayor's Office,Name - NYC Open Data Portal,Name - ODA,Name - CPO,Name - WeGov,Name - Greenbook,Name - Checkbook,Acronym,Merged Names,ID
0,actuary office of,"Actuary, NYC Office of the (NYCOA)","Actuary, NYC Office of the (NYCOA)",None,Office of the Actuary,Office of the Actuary,Office of the Actuary,"Actuary, Office of",Office of the Actuary,NYCOA,actuary office of,1
1,administration for childrens services,"Children's Services, Administration for (ACS)","Children's Services, Administration for (ACS)",Administration for Children’s Services (ACS),Administration for Children's Services (ACS),Administration for Children's Services,Administration for Children's Services,"Children's Services, Administration for",Administration for Children's Services,ACS,administration for childrens services,2
2,association for a better new york,None,None,None,Association for a Better New York (ABNY),None,None,None,None,ABNY,association for a better new york,3
3,board of correction,"Correction, Board of (BOC)","Correction, Board of (BOC)",None,Board of Correction (BOC),Board of Correction,Board of Correction,"Correction, Board of",Board of Correction,BOC,board of correction,4
4,board of elections,"Elections, Board of (BOE)","Elections, Board of (BOE)",Board of Elections (BOENY),Board of Elections (BOE),None,Board of Elections,"Elections, Board of",Board of Elections,BOE,board of elections,5


In [ ]:
deduplicated_df.shape

(337, 12)

# Filtering Out of Scope Entities

- Flagging entities that exist as multiple administrative unites based on geography (such as community boards and distrcit attorneys, keeping just one record for "Community Boards")

- Flagging non-New York City entities that are out of scope for this excersise (like new york state courts)

In [ ]:
# Add a new column "Instance Of" and initialize with None
deduplicated_df['Instance Of'] = None

# Define lists of names for different categories
community_boards = ["bronx community board", "brooklyn community board", "manhattan community board", "queens community board", "staten island community board"]
borough_presidents = ["bronx borough president", "brooklyn borough president", "manhattan borough president", "queens borough president", "staten island borough president"]
district_attorney = ["district attorney bronx county", "district attorney kings county", "district attorney new york county", "district attorney queens county", "district attorney richmond county"]
public_administrator = ["public administrator bronx county", "public administrator kings county", "public administrator new york county", "public administrator queens county", "public administrator richmond county"]

# Assign values to the "Instance Of" column based on the Name column
deduplicated_df.loc[deduplicated_df['Name'].str.lower().str.startswith(tuple(community_boards)), 'Instance Of'] = 'community boards'
deduplicated_df.loc[deduplicated_df['Name'].str.lower().isin(borough_presidents), 'Instance Of'] = 'borough presidents'
deduplicated_df.loc[deduplicated_df['Name'].str.lower().isin(district_attorney), 'Instance Of'] = 'district attorney'
deduplicated_df.loc[deduplicated_df['Name'].str.lower().isin(public_administrator), 'Instance Of'] = 'public administrator'

# Create a new row for borough presidents
new_row = pd.DataFrame([{'Name': 'borough presidents', 'Instance Of': 'borough presidents'}])
deduplicated_df = pd.concat([deduplicated_df, new_row], ignore_index=True)

In [ ]:
deduplicated_df.shape

(338, 13)

In [ ]:
# Create a new column "Out of Scope" and initialize with None
deduplicated_df['Out of Scope'] = None

# Set "Out of Scope" to "Out of Scope" for rows with a non-null value in "Instance Of"
deduplicated_df.loc[deduplicated_df['Instance Of'].notnull(), 'Out of Scope'] = 'Out of Scope'

# Set "Out of Scope" to "Out of Scope" for rows where the Name starts with "new york state" or "nys"
deduplicated_df.loc[deduplicated_df['Name'].str.lower().str.startswith('new york state'), 'Out of Scope'] = 'Out of Scope'
deduplicated_df.loc[deduplicated_df['Name'].str.lower().str.startswith('nys'), 'Out of Scope'] = 'Out of Scope'

# Filter rows to create a new DataFrame "in_scope_agencies" where "Out of Scope" is null
in_scope_agencies = deduplicated_df[deduplicated_df['Out of Scope'].isnull()]

In [ ]:
in_scope_agencies.shape

(246, 14)

In [ ]:
in_scope_agencies.head()

,Name,Name - NYC.gov Agency List,Name - NYC.gov Mayor's Office,Name - NYC Open Data Portal,Name - ODA,Name - CPO,Name - WeGov,Name - Greenbook,Name - Checkbook,Acronym,Merged Names,ID,Instance Of,Out of Scope
0,actuary office of,"Actuary, NYC Office of the (NYCOA)","Actuary, NYC Office of the (NYCOA)",None,Office of the Actuary,Office of the Actuary,Office of the Actuary,"Actuary, Office of",Office of the Actuary,NYCOA,actuary office of,1.0,None,None
1,administration for childrens services,"Children's Services, Administration for (ACS)","Children's Services, Administration for (ACS)",Administration for Children’s Services (ACS),Administration for Children's Services (ACS),Administration for Children's Services,Administration for Children's Services,"Children's Services, Administration for",Administration for Children's Services,ACS,administration for childrens services,2.0,None,None
2,association for a better new york,None,None,None,Association for a Better New York (ABNY),None,None,None,None,ABNY,association for a better new york,3.0,None,None
3,board of correction,"Correction, Board of (BOC)","Correction, Board of (BOC)",None,Board of Correction (BOC),Board of Correction,Board of Correction,"Correction, Board of",Board of Correction,BOC,board of correction,4.0,None,None
4,board of elections,"Elections, Board of (BOE)","Elections, Board of (BOE)",Board of Elections (BOENY),Board of Elections (BOE),None,Board of Elections,"Elections, Board of",Board of Elections,BOE,board of elections,5.0,None,None


#Generating an initial preferred name for each agency

The format of the CPO data source seems to be closest to what appears in the City Charter, so I will use that as the base, and then do my best to guess the best preferred name for each other row. I may go back later and create a rubric for how to decide here.

This commented out code was what I initially used to create the "perferred_name" file, which in subsequent steps I used to manually choose the preferred name and to note the legal citation for each entity.

In [ ]:
# # Code to select the Preferred Name for each row

# # Add a new column "Name - Preferred" using .loc to avoid SettingWithCopyWarning
# in_scope_agencies.loc[:, 'Name - Preferred'] = ''

# # Use .loc with apply method to set "Name - Preferred" to the value in "Name - CPO" if it exists
# in_scope_agencies.loc[:, 'Name - Preferred'] = in_scope_agencies.apply(
#     lambda row: row['Name - CPO'] if pd.notnull(row['Name - CPO']) else '',
#     axis=1
# )

# # Function to prompt user to choose a value for "Name - Preferred"
# def choose_name(row):
#     options = [
#         "Name - NYC.gov Agency List",
#         "Name - NYC.gov Mayor's Office",
#         "Name - NYC Open Data Portal",
#         "Name - ODA",
#         "Name - CPO",
#         "Name - WeGov",
#         "Name - Greenbook",
#         "Name - Checkbook"
#     ]

#     print("\nChoose a preferred name for the following row:")
#     for idx, option in enumerate(options, 1):
#         value = row.get(option, '')
#         print(f"{str(idx).ljust(2)}. {option.ljust(30)} : {str(value).rjust(50)}")

#     choice = input("\nEnter the number of the preferred name (or leave blank to skip): ")
#     if choice.isdigit():
#         choice = int(choice)
#         if 1 <= choice <= len(options):
#             return row[options[choice - 1]]
#     return ''

# # Iterate over rows where "Name - Preferred" is empty
# for index, row in in_scope_agencies[in_scope_agencies['Name - Preferred'] == ''].iterrows():
#     preferred_name = choose_name(row)
#     in_scope_agencies.at[index, 'Name - Preferred'] = preferred_name

In [ ]:
# Path to the CSV file
# preferred_name = '/content/drive/MyDrive/Projects/ODA/Agency Name Project/Output/preferred_name.csv'

# Load the CSV file into a DataFrame
# preferred_name = pd.read_csv(preferred_name)

In [ ]:
# Merge in_scope_agencies with preferred_name_df to add the "Preferred Name" column based on the "Name" field
# in_scope_agencies = in_scope_agencies.merge(preferred_name[['Name', 'Name - Preferred']], on='Name', how='left')

In [ ]:
# # Function to remove acronyms and parentheses
# def strip_acronyms(name):
#     if isinstance(name, str):
#         return re.sub(r'\s*\([^)]*\)', '', name).strip()
#     return name

# # Strip acronyms from the "Name - Preferred" column
# if 'Name - Preferred' in in_scope_agencies.columns:
#     in_scope_agencies['Name - Preferred'] = in_scope_agencies['Name - Preferred'].apply(strip_acronyms)

# Merging in Manually Compiled Preferred Name, Agency Type, and Legal Citation information

The agencies_with_charter_citation.csv file was manually compiled

In [ ]:
# Path to the CSV file
agencies_with_charter_citation = '/content/drive/MyDrive/Projects/ODA/Agency Name Project/Output/agencies_with_charter_citation.csv'

# Load the CSV file into a DataFrame
agencies_with_charter_citation = pd.read_csv(agencies_with_charter_citation)

In [ ]:
agencies_with_charter_citation.shape

(293, 18)

In [ ]:
in_scope_agencies.shape

(246, 14)

In [ ]:
#agencies_enhanced = pd.merge(in_scope_agencies, agencies_with_charter_citation, on="Name", how="left")
agencies_enhanced = pd.merge(in_scope_agencies, agencies_with_charter_citation, on="Name", how="outer", indicator=True)

In [ ]:
agencies_enhanced.shape

(298, 32)

In [ ]:
agencies_enhanced.columns

Index(['Name', 'Name - NYC.gov Agency List', 'Name - NYC.gov Mayor's Office',
       'Name - NYC Open Data Portal', 'Name - ODA', 'Name - CPO',
       'Name - WeGov', 'Name - Greenbook', 'Name - Checkbook', 'Acronym',
       'Merged Names', 'ID', 'Instance Of', 'Out of Scope', 'Name - Preferred',
       'NYC Administrative Organization Type', 'Agency Operational Status',
       'Parent Organization', 'Authorizing Authority', 'Legal Citation',
       'Legal Citation URL', 'Legal Citation Text', 'Legal Name',
       'Current Operating Name', 'Alias', 'Acronym - Preferred',
       'Alternate Acronym', 'Data Notes', 'Manual Addition Flag',
       'Manual Addition Source', 'Website', '_merge'],
      dtype='object')

In [ ]:
# Get distinct values from 'Agency Operational Status' field
distinct_values = agencies_enhanced['Agency Operational Status'].unique()

# Display the distinct values
print(distinct_values)

['Active' 'Probably Defunct' 'Duplicate' 'Unable to locate' 'Reorganized'
 'Inactive' 'Policy' nan 'Delete' 'Out of Scope' 'Unable to find citation']


Filter for just Active Agencies

In [ ]:
# Define the list of desired statuses
filtered_statuses = ['Active', 'Disbanded', 'Renamed', 'Reorganized', 'Out of Scope']

# Filter the DataFrame based on the desired statuses
filtered_agencies = agencies_enhanced[agencies_enhanced['Agency Operational Status'].isin(filtered_statuses)]

In [ ]:
agencies_enhanced.shape

(298, 32)

In [ ]:
filtered_agencies.shape

(264, 32)

In [ ]:
agencies_enhanced = filtered_agencies

In [ ]:
agencies_enhanced.shape

(264, 32)

In [ ]:
# Merge the DataFrames on 'Name - NYC.gov Agency List'
agency_name_final = agencies_enhanced.merge(df_nyc_gov_agency_list[['Name - NYC.gov Agency List', 'URL']],
                                           on='Name - NYC.gov Agency List',
                                           how='left')

# Rename the 'URL' column to 'Organization Website'
agency_name_final.rename(columns={'URL': 'Organization Website'}, inplace=True)

In [ ]:
df_nyc_mayor_office.columns

Index(['Name', 'Name - NYC.gov Mayor's Office', 'URL', 'Contact Name',
       'Contact Title'],
      dtype='object')

In [ ]:
df_oda_data.columns

Index(['Name', 'Agency Code', 'Parent Organization', 'Child Organization(s)',
       'Acronym', 'Agency Type', 'Website', 'Name - ODA'],
      dtype='object')

In [ ]:
agencies_enhanced.columns

Index(['Name', 'Name - NYC.gov Agency List', 'Name - NYC.gov Mayor's Office',
       'Name - NYC Open Data Portal', 'Name - ODA', 'Name - CPO',
       'Name - WeGov', 'Name - Greenbook', 'Name - Checkbook', 'Acronym',
       'Merged Names', 'ID', 'Instance Of', 'Out of Scope', 'Name - Preferred',
       'NYC Administrative Organization Type', 'Agency Operational Status',
       'Parent Organization', 'Authorizing Authority', 'Legal Citation',
       'Legal Citation URL', 'Legal Citation Text', 'Legal Name',
       'Current Operating Name', 'Alias', 'Acronym - Preferred',
       'Alternate Acronym', 'Data Notes', 'Manual Addition Flag',
       'Manual Addition Source', 'Website', '_merge'],
      dtype='object')

In [ ]:
# Merge the DataFrames on 'Name - NYC.gov Agency List' and keep all original columns
agency_name_final = agencies_enhanced.merge(df_nyc_gov_agency_list[['Name - NYC.gov Agency List', 'URL', 'Description']],
                                            on='Name - NYC.gov Agency List',
                                            how='left')

# Rename the 'URL' column to 'Organization Website'
agency_name_final.rename(columns={'URL': 'Organization Website'}, inplace=True)

# Merge with df_nyc_mayor_office to fill in missing 'Organization Website' and add 'Contact Name'
agency_name_final = agency_name_final.merge(df_nyc_mayor_office[['Name - NYC.gov Mayor\'s Office', 'URL', 'Contact Name']],
                                            left_on='Name - NYC.gov Mayor\'s Office',
                                            right_on='Name - NYC.gov Mayor\'s Office',
                                            how='left')

# Fill in missing 'Organization Website' values from the Mayor's Office data
agency_name_final['Organization Website'].fillna(agency_name_final['URL'], inplace=True)

# Merge with df_oda_data to fill in remaining missing 'Organization Website'
agency_name_final = agency_name_final.merge(df_oda_data[['Name - ODA', 'Website']],
                                            left_on='Name - NYC.gov Agency List',
                                            right_on='Name - ODA',
                                            how='left')

In [ ]:
agency_name_final.head()

,Name,Name - NYC.gov Agency List,Name - NYC.gov Mayor's Office,Name - NYC Open Data Portal,Name - ODA_x,Name - CPO,Name - WeGov,Name - Greenbook,Name - Checkbook,Acronym,...,Manual Addition Flag,Manual Addition Source,Website_x,_merge,Organization Website,Description,URL,Contact Name,Name - ODA_y,Website_y
0,actuary office of,"Actuary, NYC Office of the (NYCOA)","Actuary, NYC Office of the (NYCOA)",None,Office of the Actuary,Office of the Actuary,Office of the Actuary,"Actuary, Office of",Office of the Actuary,NYCOA,...,NaN,NaN,NaN,both,http://www.nyc.gov/actuary,"The New York City Office of the Actuary (""NYCO...",http://www.nyc.gov/actuary,Marek Tyszkiewicz,NaN,NaN
1,administration for childrens services,"Children's Services, Administration for (ACS)","Children's Services, Administration for (ACS)",Administration for Children’s Services (ACS),Administration for Children's Services (ACS),Administration for Children's Services,Administration for Children's Services,"Children's Services, Administration for",Administration for Children's Services,ACS,...,NaN,NaN,NaN,both,http://www.nyc.gov/acs,The Administration for Children's Services (AC...,http://www.nyc.gov/acs,Jess Dannhauser,NaN,NaN
2,association for a better new york,None,None,None,Association for a Better New York (ABNY),None,None,None,None,ABNY,...,NaN,NaN,NaN,both,NaN,NaN,NaN,NaN,NaN,NaN
3,board of correction,"Correction, Board of (BOC)","Correction, Board of (BOC)",None,Board of Correction (BOC),Board of Correction,Board of Correction,"Correction, Board of",Board of Correction,BOC,...,NaN,NaN,NaN,both,http://www.nyc.gov/boc,The NYC Board of Correction establishes and en...,http://www.nyc.gov/boc,Jasmine Georges-Villa,NaN,NaN
4,board of elections,"Elections, Board of (BOE)","Elections, Board of (BOE)",Board of Elections (BOENY),Board of Elections (BOE),None,Board of Elections,"Elections, Board of",Board of Elections,BOE,...,NaN,NaN,NaN,both,https://vote.nyc/,The Board of Elections is an administrative bo...,https://vote.nyc/,Simon Shamoun,NaN,NaN


In [ ]:
# Fill in missing 'Organization Website' values
agency_name_final['Organization Website'] = agency_name_final['Organization Website'].fillna(agency_name_final['Website_x']).fillna(agency_name_final['Website_y'])

In [ ]:
# Fill in missing 'Organization Website' values from the ODA data
#agency_name_final['Organization Website'].fillna(agency_name_final['Website'], inplace=True)

# Drop temporary columns
agency_name_final.drop(columns=['URL', 'Website'], errors='ignore', inplace=True)
agency_name_final.rename(columns={'Name - ODA_x': 'Name - ODA'}, inplace=True)

# Drop columns ending with '_x' and '_y'
agency_name_final = agency_name_final.loc[:, ~agency_name_final.columns.str.endswith(('_x', '_y'))]

In [ ]:
agency_name_final.columns

Index(['Name', 'Name - NYC.gov Agency List', 'Name - NYC.gov Mayor's Office',
       'Name - NYC Open Data Portal', 'Name - ODA', 'Name - CPO',
       'Name - WeGov', 'Name - Greenbook', 'Name - Checkbook', 'Acronym',
       'Merged Names', 'ID', 'Instance Of', 'Out of Scope', 'Name - Preferred',
       'NYC Administrative Organization Type', 'Agency Operational Status',
       'Parent Organization', 'Authorizing Authority', 'Legal Citation',
       'Legal Citation URL', 'Legal Citation Text', 'Legal Name',
       'Current Operating Name', 'Alias', 'Acronym - Preferred',
       'Alternate Acronym', 'Data Notes', 'Manual Addition Flag',
       'Manual Addition Source', '_merge', 'Organization Website',
       'Description', 'Contact Name'],
      dtype='object')

In [ ]:
df_nyc_gov_agency_list.columns

Index(['Name', 'Name - NYC.gov Agency List', 'URL', 'Description'], dtype='object')

In [ ]:
df_nyc_mayor_office.columns

Index(['Name', 'Name - NYC.gov Mayor's Office', 'URL', 'Contact Name',
       'Contact Title'],
      dtype='object')

In [ ]:
# Merge to add the 'Description' column
agency_name_final = agency_name_final.merge(df_nyc_gov_agency_list[['Name - NYC.gov Agency List', 'Description']],
                                            on='Name - NYC.gov Agency List',
                                            how='left')

In [ ]:
# Merge to add the 'Contact Name' column
agency_name_final = agency_name_final.merge(df_nyc_mayor_office[['Name - NYC.gov Mayor\'s Office', 'Contact Name', 'Contact Title']],
                                            on='Name - NYC.gov Mayor\'s Office',
                                            how='left')

In [ ]:
# Merge to add the 'Open Datasets URL' column
agency_name_final = agency_name_final.merge(df_nyc_open_data_portal[['Name - NYC Open Data Portal', 'URL']],
                                            left_on='Name - NYC Open Data Portal',
                                            right_on='Name - NYC Open Data Portal',
                                            how='left')

# Rename the 'URL' column to 'Open Datasets URL'
agency_name_final.rename(columns={'URL': 'Open Datasets URL'}, inplace=True)

In [ ]:
agency_name_final.columns

Index(['Name', 'Name - NYC.gov Agency List', 'Name - NYC.gov Mayor's Office',
       'Name - NYC Open Data Portal', 'Name - ODA', 'Name - CPO',
       'Name - WeGov', 'Name - Greenbook', 'Name - Checkbook', 'Acronym',
       'Merged Names', 'ID', 'Instance Of', 'Out of Scope', 'Name - Preferred',
       'NYC Administrative Organization Type', 'Agency Operational Status',
       'Parent Organization', 'Authorizing Authority', 'Legal Citation',
       'Legal Citation URL', 'Legal Citation Text', 'Legal Name',
       'Current Operating Name', 'Alias', 'Acronym - Preferred',
       'Alternate Acronym', 'Data Notes', 'Manual Addition Flag',
       'Manual Addition Source', '_merge', 'Organization Website',
       'Description_x', 'Contact Name_x', 'Description_y', 'Contact Name_y',
       'Contact Title', 'Open Datasets URL'],
      dtype='object')

In [ ]:
# Merge to add the 'Agency Code' column
agency_name_final = agency_name_final.merge(df_oda_data[['Name - ODA', 'Agency Code']],
                                            left_on='Name - ODA',
                                            right_on='Name - ODA',
                                            how='left')

In [ ]:
agency_name_final.head(2)

,Name,Name - NYC.gov Agency List,Name - NYC.gov Mayor's Office,Name - NYC Open Data Portal,Name - ODA,Name - CPO,Name - WeGov,Name - Greenbook,Name - Checkbook,Acronym,...,Manual Addition Source,_merge,Organization Website,Description_x,Contact Name_x,Description_y,Contact Name_y,Contact Title,Open Datasets URL,Agency Code
0,actuary office of,"Actuary, NYC Office of the (NYCOA)","Actuary, NYC Office of the (NYCOA)",None,Office of the Actuary,Office of the Actuary,Office of the Actuary,"Actuary, Office of",Office of the Actuary,NYCOA,...,NaN,both,http://www.nyc.gov/actuary,"The New York City Office of the Actuary (""NYCO...",Marek Tyszkiewicz,"The New York City Office of the Actuary (""NYCO...",Marek Tyszkiewicz,Chief Actuary,NaN,008
1,administration for childrens services,"Children's Services, Administration for (ACS)","Children's Services, Administration for (ACS)",Administration for Children’s Services (ACS),Administration for Children's Services (ACS),Administration for Children's Services,Administration for Children's Services,"Children's Services, Administration for",Administration for Children's Services,ACS,...,NaN,both,http://www.nyc.gov/acs,The Administration for Children's Services (AC...,Jess Dannhauser,The Administration for Children's Services (AC...,Jess Dannhauser,Commissioner,https://data.cityofnewyork.us/browse?Dataset-I...,068


In [ ]:
agency_name_final.columns

Index(['Name', 'Name - NYC.gov Agency List', 'Name - NYC.gov Mayor's Office',
       'Name - NYC Open Data Portal', 'Name - ODA', 'Name - CPO',
       'Name - WeGov', 'Name - Greenbook', 'Name - Checkbook', 'Acronym',
       'Merged Names', 'ID', 'Instance Of', 'Out of Scope', 'Name - Preferred',
       'NYC Administrative Organization Type', 'Agency Operational Status',
       'Parent Organization', 'Authorizing Authority', 'Legal Citation',
       'Legal Citation URL', 'Legal Citation Text', 'Legal Name',
       'Current Operating Name', 'Alias', 'Acronym - Preferred',
       'Alternate Acronym', 'Data Notes', 'Manual Addition Flag',
       'Manual Addition Source', '_merge', 'Organization Website',
       'Description_x', 'Contact Name_x', 'Description_y', 'Contact Name_y',
       'Contact Title', 'Open Datasets URL', 'Agency Code'],
      dtype='object')

In [ ]:
# Create a new DataFrame called nyc_agencies based on agency_name_final
nyc_agencies = agency_name_final.copy()

# Rename 'Name' to 'NameID'
nyc_agencies.rename(columns={'Name': 'NameID'}, inplace=True)

# Drop the specified fields, excluding 'Name' which is now 'NameID'
nyc_agencies.drop(columns=['Description_y', 'Contact Name_y'], errors='ignore', inplace=True)

In [ ]:
# Rename the specified fields
nyc_agencies.rename(columns={'Description_x': 'Description', 'Contact Name_x': 'Contact Name'}, inplace=True)

In [ ]:
nyc_agencies.columns

Index(['NameID', 'Name - NYC.gov Agency List', 'Name - NYC.gov Mayor's Office',
       'Name - NYC Open Data Portal', 'Name - ODA', 'Name - CPO',
       'Name - WeGov', 'Name - Greenbook', 'Name - Checkbook', 'Acronym',
       'Merged Names', 'ID', 'Instance Of', 'Out of Scope', 'Name - Preferred',
       'NYC Administrative Organization Type', 'Agency Operational Status',
       'Parent Organization', 'Authorizing Authority', 'Legal Citation',
       'Legal Citation URL', 'Legal Citation Text', 'Legal Name',
       'Current Operating Name', 'Alias', 'Acronym - Preferred',
       'Alternate Acronym', 'Data Notes', 'Manual Addition Flag',
       'Manual Addition Source', '_merge', 'Organization Website',
       'Description', 'Contact Name', 'Contact Title', 'Open Datasets URL',
       'Agency Code'],
      dtype='object')

In [ ]:
# Define the mapping dictionary based on the provided CSV, including the missing fields
mapping = {
    'NameID': 'NameID',
    'Name - Preferred': 'Name',
    'Agency Operational Status': 'OperationalStatus',
    'NYC Administrative Organization Type': 'OrganizationType',
    'Description': 'Description',
    'Organization Website': 'URL',
    'Parent Organization': 'ParentOrganization',
    'Authorizing Authority': 'AuthorizingAuthority',
    'Legal Citation': 'LegalCitation',
    'Legal Citation URL': 'LegalCitationURL',
    'Legal Citation Text': 'LegalCitationText',
    'Legal Name': 'LegalName',
    'Current Operating Name': 'CurrentOperatingName',
    'Alias': 'AlternateNames',
    'Acronym': 'Acronym',
#    'Acronym - Preferred': 'Acronym',
    'Alternate Acronym': 'AlternateAcronyms',
    'Agency Code': 'BudgetCode',
    'Contact Name': 'PrincipalOfficerName',
    'Contact Title': 'PrincipalOfficerTitle',
    'Open Datasets URL': 'OpenDatasetsURL',
    'Data Notes': 'Notes',
    'Manual Addition Flag': 'ManualAdditionFlag',
    'Manual Addition Source': 'ManualAdditionSource'
}

# List of new fields to add
new_fields = [
    'Jurisdiction',
    'AdministrativeLevel',
    'FoundingDate',
    'SunsetDate',
    'SuccessorOrganization',
    'URISlug',
    'DateCreated',
    'DateModified',
    'LastVerifiedDate'
]

# Rename the columns
nyc_agencies = nyc_agencies.rename(columns=mapping)

# Add new fields with null values
for field in new_fields:
    nyc_agencies[field] = pd.NA

# Get the list of columns that were not renamed and are not in the new fields
unchanged_columns = [col for col in nyc_agencies.columns if col not in mapping.keys() and col not in mapping.values() and col not in new_fields]

# Create the new column order: renamed columns first, then new fields, then unchanged columns
new_column_order = list(mapping.values()) + new_fields + unchanged_columns

# Reorder the columns
nyc_agencies = nyc_agencies[new_column_order]

# Display the new column names
#print(nyc_agencies.columns.tolist())

In [ ]:
nyc_agencies.columns

Index(['NameID', 'Name', 'OperationalStatus', 'OrganizationType',
       'Description', 'URL', 'ParentOrganization', 'AuthorizingAuthority',
       'LegalCitation', 'LegalCitationURL', 'LegalCitationText', 'LegalName',
       'CurrentOperatingName', 'AlternateNames', 'Acronym',
       'AlternateAcronyms', 'BudgetCode', 'PrincipalOfficerName',
       'PrincipalOfficerTitle', 'OpenDatasetsURL', 'Notes',
       'ManualAdditionFlag', 'ManualAdditionSource', 'Jurisdiction',
       'AdministrativeLevel', 'FoundingDate', 'SunsetDate',
       'SuccessorOrganization', 'URISlug', 'DateCreated', 'DateModified',
       'LastVerifiedDate', 'Name - NYC.gov Agency List',
       'Name - NYC.gov Mayor's Office', 'Name - NYC Open Data Portal',
       'Name - ODA', 'Name - CPO', 'Name - WeGov', 'Name - Greenbook',
       'Name - Checkbook', 'Merged Names', 'ID', 'Instance Of', 'Out of Scope',
       'Acronym - Preferred', '_merge'],
      dtype='object')

In [ ]:
# List of columns to be removed
columns_to_remove = ['Merged Names', 'ID', 'Instance Of', 'Out of Scope', 'ManualAdditionFlag', 'ManualAdditionSource', 'Acronym - Preferred']

# Remove the specified columns
nyc_agencies.drop(columns=columns_to_remove, inplace=True)

In [ ]:
nyc_agencies.columns

Index(['NameID', 'Name', 'OperationalStatus', 'OrganizationType',
       'Description', 'URL', 'ParentOrganization', 'AuthorizingAuthority',
       'LegalCitation', 'LegalCitationURL', 'LegalCitationText', 'LegalName',
       'CurrentOperatingName', 'AlternateNames', 'Acronym',
       'AlternateAcronyms', 'BudgetCode', 'PrincipalOfficerName',
       'PrincipalOfficerTitle', 'OpenDatasetsURL', 'Notes', 'Jurisdiction',
       'AdministrativeLevel', 'FoundingDate', 'SunsetDate',
       'SuccessorOrganization', 'URISlug', 'DateCreated', 'DateModified',
       'LastVerifiedDate', 'Name - NYC.gov Agency List',
       'Name - NYC.gov Mayor's Office', 'Name - NYC Open Data Portal',
       'Name - ODA', 'Name - CPO', 'Name - WeGov', 'Name - Greenbook',
       'Name - Checkbook', '_merge'],
      dtype='object')

In [ ]:
filtered_nyc_agencies.shape

(51, 39)

In [ ]:
agency_name_crosswalk = nyc_agencies[[
    'Name',
    'Acronym',
    'BudgetCode',
    'OrganizationType',
    'OperationalStatus',
    'Name - NYC.gov Agency List',
    'Name - NYC.gov Mayor\'s Office',
    'Name - NYC Open Data Portal',
    'Name - ODA',
    'Name - CPO',
    'Name - WeGov',
    'Name - Greenbook',
    'Name - Checkbook'
]].copy()

In [ ]:
agency_name_crosswalk.shape

(264, 13)

In [ ]:
nyc_agencies.shape

(264, 39)

In [ ]:
nyc_agencies_qa2 = nyc_agencies

#Updating nyc_agencies_qa2 with the manual updates I made to nyc_agencies_qa
I need to do this because nyc_agencies_qa was created with a left join instead of an outer join so it's missing some rows. After I do this I will download nyc_agencies_qa2 and continue the manual QA process from that file.

In [ ]:
# First, load your manually QA'd file
nyc_agencies_qa = pd.read_csv('/content/drive/MyDrive/Projects/ODA/Agency Name Project/Output/nyc_agencies_qa.csv')

# Verify that all columns (except '_merge') are the same in both DataFrames
qa_columns = set(nyc_agencies_qa.columns)
qa2_columns = set(nyc_agencies_qa2.columns) - {'_merge'}

if qa_columns != qa2_columns:
    print("Warning: The columns in the two DataFrames do not match exactly.")
    print("Columns only in nyc_agencies_qa:", qa_columns - qa2_columns)
    print("Columns only in nyc_agencies_qa2:", qa2_columns - qa_columns)
else:
    print("All columns match between the two DataFrames (excluding '_merge').")

All columns match between the two DataFrames (excluding '_merge').


In [ ]:
# Create a mask for matching 'Name' values
mask = nyc_agencies_qa2['Name'].isin(nyc_agencies_qa['Name'])

# Check if all other fields match for the rows with matching 'Name'
columns_to_check = list(qa_columns - {'Name'})
mismatched_rows = []

for index, row in nyc_agencies_qa2[mask].iterrows():
    qa_row = nyc_agencies_qa[nyc_agencies_qa['Name'] == row['Name']].iloc[0]
    if not all((pd.isna(row[col]) and pd.isna(qa_row[col])) or (row[col] == qa_row[col]) for col in columns_to_check):
        mismatched_rows.append(row['Name'])

if mismatched_rows:
    print("Warning: The following rows have matching 'Name' but mismatched values in other columns:")
    print(mismatched_rows)
    proceed = input("Do you want to proceed with updating these rows? (yes/no): ")
    if proceed.lower() != 'yes':
        print("Operation aborted.")
        exit()

# Update nyc_agencies_qa2 with values from nyc_agencies_qa where 'Name' matches
columns_to_update = list(qa_columns)
nyc_agencies_qa2.loc[mask, columns_to_update] = nyc_agencies_qa[nyc_agencies_qa['Name'].isin(nyc_agencies_qa2['Name'])][columns_to_update].values

print("Update complete.")

# Display some information about the update
print(f"Total rows in nyc_agencies_qa2: {len(nyc_agencies_qa2)}")
print(f"Rows updated: {mask.sum()}")
print(f"Rows not updated (potentially new agencies): {(~mask).sum()}")

# If you want to see the rows that were not updated (potentially new agencies):
new_agencies = nyc_agencies_qa2[~mask]
print("\nPotentially new agencies:")
print(new_agencies[['Name', '_merge']])

['New York City Office of the Actuary', "Administration for Children's Services", 'Association for a Better New York', 'Board of Correction', 'Board of Elections', 'Board of Standards and Appeals', 'Borough Historians', 'Borough Historical Societies', 'Broadway Theater', 'Brooklyn Bridge Park Corporation', 'Brooklyn Public Library', 'Department of Buildings', 'Business Integrity Commission', 'Campaign Finance Board', 'Center for Innovation through Data Intelligence', 'Charter Revision Commission', 'Chief Efficiency Officer', 'Chief Housing Officer', 'City Council', "Mayor's Office of City Legislative Affairs", 'Department of City Planning', 'City Planning Commission', 'City University Construction Fund', 'City University of New York', 'City University of New York - Board of Trustees', "Mayor's Office of Citywide Event Coordination and Management", 'Civil Service Commission', 'Civilian Complaint Review Board', 'Office of Collective Bargaining', "Commission on Women's Issues", "Mayor's C

In [ ]:
nyc_agencies_qa2.shape

(264, 39)

In [ ]:
# Check for duplicates based on the 'Name' field
duplicate_mask = nyc_agencies_qa2.duplicated(subset='Name', keep=False)

# Add a new column '_PotentialDuplicate' and mark duplicates
nyc_agencies_qa2['_PotentialDuplicate'] = duplicate_mask

# Print summary of duplicates found
num_duplicates = duplicate_mask.sum()
print(f"Number of potential duplicates found: {num_duplicates}")

# If duplicates are found, display them
if num_duplicates > 0:
    print("\nPotential duplicates:")
    print(nyc_agencies_qa2[duplicate_mask][['Name', '_merge', '_PotentialDuplicate']])

# Optional: Save the updated DataFrame to a new CSV file
# nyc_agencies_qa2.to_csv('nyc_agencies_qa2_with_duplicates.csv', index=False)

# Display the first few rows of the updated DataFrame to verify the new column
print("\nFirst few rows of the updated DataFrame:")
print(nyc_agencies_qa2[['Name', '_merge', '_PotentialDuplicate']].head())

Number of potential duplicates found: 10

Potential duplicates:
                                                  Name      _merge  \
40   Office of the Chief Counsel to the Mayor and C...        both   
150                        Commission on Racial Equity        both   
172  Office of the Chief Counsel to the Mayor and C...        both   
176                     Office to Prevent Gun Violence        both   
177                     Office to Prevent Gun Violence        both   
190                        Commission on Racial Equity        both   
213     Department of Citywide Administrative Services  right_only   
214     Department of Citywide Administrative Services  right_only   
216                   Office of Payroll Administration  right_only   
217                   Office of Payroll Administration  right_only   

     _PotentialDuplicate  
40                  True  
150                 True  
172                 True  
176                 True  
177                 True  
190

In [ ]:
nyc_agencies_qa2.columns

Index(['NameID', 'Name', 'OperationalStatus', 'OrganizationType',
       'Description', 'URL', 'ParentOrganization', 'AuthorizingAuthority',
       'LegalCitation', 'LegalCitationURL', 'LegalCitationText', 'LegalName',
       'CurrentOperatingName', 'AlternateNames', 'Acronym',
       'AlternateAcronyms', 'BudgetCode', 'PrincipalOfficerName',
       'PrincipalOfficerTitle', 'OpenDatasetsURL', 'Notes', 'Jurisdiction',
       'AdministrativeLevel', 'FoundingDate', 'SunsetDate',
       'SuccessorOrganization', 'URISlug', 'DateCreated', 'DateModified',
       'LastVerifiedDate', 'Name - NYC.gov Agency List',
       'Name - NYC.gov Mayor's Office', 'Name - NYC Open Data Portal',
       'Name - ODA', 'Name - CPO', 'Name - WeGov', 'Name - Greenbook',
       'Name - Checkbook', '_merge', '_PotentialDuplicate'],
      dtype='object')

Change the formatting of the BudgetCode field

In [ ]:
# Function to pad budget code with leading zeros
def pad_budget_code(code):
    if pd.isna(code):  # Check if the value is NaN
        return code
    try:
        # Try to convert to integer and then format as a 3-digit string
        return f"{int(code):03d}"
    except ValueError:
        # If conversion to int fails, return the original value
        return code

# Apply the padding function to the BudgetCode column
nyc_agencies_qa2['BudgetCode'] = nyc_agencies_qa2['BudgetCode'].apply(pad_budget_code)

# Display a sample of the updated BudgetCode column
print(nyc_agencies_qa2[['Name', 'BudgetCode']].sample(10))

# Optional: Count of unique values in BudgetCode column
print("\nNumber of unique BudgetCode values:", nyc_agencies_qa2['BudgetCode'].nunique())

# Optional: Display rows where BudgetCode is not a 3-digit string (to check for any anomalies)
non_standard_codes = nyc_agencies_qa2[~nyc_agencies_qa2['BudgetCode'].str.match(r'^\d{3}$', na=True)]
if not non_standard_codes.empty:
    print("\nRows with non-standard BudgetCode values:")
    print(non_standard_codes[['Name', 'BudgetCode']])
else:
    print("\nAll BudgetCode values are now in the standard 3-digit format or NaN.")

                                              Name BudgetCode
232            Office of Street Vendor Enforcement        NaN
65           Equal Employment Practices Commission        133
34                                Community Boards        NaN
238            Office of Healthcare Accountability        NaN
115    Mayor's Office of Environmental Remediation        NaN
5                   Board of Standards and Appeals        NaN
120  Mayor's Office of Housing Recovery Operations        002
35                      Office of Community Hiring        NaN
209                       Union Square Partnership        NaN
146                    New York Research Libraries        NaN

Number of unique BudgetCode values: 57

All BudgetCode values are now in the standard 3-digit format or NaN.


# Export QA2 file to CSV

In [ ]:
# Define the output file path
output_file_path = '/content/drive/MyDrive/Projects/ODA/Agency Name Project/Output/nyc_agencies_qa2.csv'

# Export the dataframe to a CSV file
nyc_agencies_qa2.to_csv(output_file_path, index=False)

print(f'Data exported successfully to {output_file_path}')

Data exported successfully to /content/drive/MyDrive/Projects/ODA/Agency Name Project/Output/nyc_agencies_qa2.csv


# Upload Manually QAed nyc_agencies_qa2 file

In [ ]:
# # File path
# file_path = '/content/drive/MyDrive/Projects/ODA/Agency Name Project/Output/nyc_agencies_qa2.csv'

# # Read the CSV file
# nyc_agencies_final = pd.read_csv(file_path)

In [ ]:
# nyc_agencies_final.shape

(276, 42)

#Scrape press contact info from https://www.nyc.gov/nyc-resources/agency-press-contacts.page

This is being commentted out as it will become a stand alone dataset. Press contacts are a specific kind of contact, and including them in this general dataset might confuse some users looking for a general contact for agencies.

In [ ]:
# pip install selenium webdriver_manager pandas

In [ ]:
# import re
# import pandas as pd

# def parse_press_contacts(js_content):
#     pattern = r'press\[press\.length\]\s*=\s*\[(.*?)\];'
#     entries = re.findall(pattern, js_content, re.DOTALL)

#     agencies = []
#     for entry in entries:
#         # Split the entry by comma, but not within quotes
#         data = re.findall(r'"([^"]*)"', entry)

#         agency_data = {
#             'AgencyName': data[0],
#             'PressContactName': data[1] if len(data) > 1 else '',
#             'PressContactTitle': data[2] if len(data) > 2 else '',
#             'PressContactEmail': data[3] if len(data) > 3 else '',
#             'PressContactPhone': data[4] if len(data) > 4 else '',
#             'GeneralInfoEmail': '',
#             'GeneralInfoPhone': ''
#         }

#         if 'General Info' in agency_data['PressContactName']:
#             agency_data['GeneralInfoEmail'] = agency_data['PressContactEmail']
#             agency_data['GeneralInfoPhone'] = agency_data['PressContactPhone']
#             agency_data['PressContactName'] = ''
#             agency_data['PressContactTitle'] = ''
#             agency_data['PressContactEmail'] = ''
#             agency_data['PressContactPhone'] = ''

#         agencies.append(agency_data)

#     return pd.DataFrame(agencies)

# # JavaScript content (paste the entire content here)
# js_content = """
# var press = new Array();
# press[press.length] = ["Actuary", "Markoe-Boyd, Marlene", "Director of Communications", "mmarkoe-boyd@actuary.nyc.gov", "212-312-0119"];
# press[press.length] = ["Aging", "General Info", "", "media@aging.nyc.gov", "212-602-4152"];
# press[press.length] = ["Buildings", "Rudansky, Andrew", "Press Secretary", "ARudansky@buildings.nyc.gov", "212-393-2072"];
# press[press.length] = ["Buildings", "General Info", "", "DOBCommunications@buildings.nyc.gov", "212-393-2126"];
# press[press.length] = ["Chief Medical Examiner", "Bolcer, Julie", "Executive Director of Public Affairs and Senior Advisor", "jbolcer@ocme.nyc.gov", "212-447-2041"];
# press[press.length] = ["Chief Medical Examiner", "General Info", "", "publicaffairs@ocme.nyc.gov", "212-447-2041"];
# press[press.length] = ["Children's Services", "Kaufman, Marisa", "Assistant Commissioner for Communications", "Marisa.Kaufman@acs.nyc.gov", "212-341-2689"];
# press[press.length] = ["Children's Services", "General Info", "", "mediainquiry@acs.nyc.gov", ""];
# press[press.length] = ["City Planning", "General info", "", "press@planning.nyc.gov", "212-720-3471"];
# press[press.length] = ["Citywide Administrative Services", "Hodgson, Anessa", "Deputy Communications Director", "ahodgson@dcas.nyc.gov", "212-386-0215"];
# press[press.length] = ["Civilian Complaint Review Board", "Calvo-Platero, Clio", "Senior Communications Advisor", "ccalvo-platero@ccrb.nyc.gov", "917-224-0422"];
# press[press.length] = ["Climate and Environmental Justice", "Winston, Kimberly", "Director of Communications", "kwinston@climate.nyc.gov", "212-676-3031"];
# press[press.length] = ["Climate and Environmental Justice", "General Info", "", "MOCEJPress@cityhall.nyc.gov", ""];
# press[press.length] = ["Consumer and Worker Protection", "Lanza, Michael", "Press Secretary", "mlanza@dcwp.nyc.gov", "212-908-4967"];
# press[press.length] = ["Consumer and Worker Protection", "DCWP Press Office ", "", "press@dcwp.nyc.gov", "212-436-0042"];
# press[press.length] = ["Contract Services", "Geiling, Jennifer", "Press Officer/Deputy Director - Partnerships", "Jennifer.Geiling@mocs.nyc.gov", "212-298-0603"];
# press[press.length] = ["Correction", "Contact the Mayor's Press Office", "", "pressoffice@cityhall.nyc.gov", "212-788-2958"];
# press[press.length] = ["Criminal Justice", "Contact the Mayor's Press Office", "", "pressoffice@cityhall.nyc.gov", "212-788-2958"];
# press[press.length] = ["Cultural Affairs", "Max, Ryan", "Director of External Affairs", "rmax@culture.nyc.gov", "212-513-9323"];
# press[press.length] = ["Cultural Affairs", "General Info", "", "publicaffairs@culture.nyc.gov", ""];
# press[press.length] = ["Design & Construction", "Michaels, Ian", "Executive Director of Public Information", "michaelia@ddc.nyc.gov", "718-391-1589"];
# press[press.length] = ["Economic Development Corporation", "General Info", "", "publicaffairs@edc.nyc", ""];
# press[press.length] = ["Education", "Styer, Nathaniel", "Press Secretary", "NStyer@schools.nyc.gov", "616-403-4693"];
# press[press.length] = ["Education", "General Info", "", "communications@schools.nyc.gov", ""];
# press[press.length] = ["Emergency Management", "Holmes, Ashleigh", "Senior Press Officer, Community and Ethnic Media", "aholmes@oem.nyc.gov", "718-422-4888"];
# press[press.length] = ["Emergency Management", "General Info", "", "press@oem.nyc.gov", ""];
# press[press.length] = ["Environmental Protection", "Timbers, Ted", "Director of Communications", "etimbers@dep.nyc.gov", "718-595-6600"];
# press[press.length] = ["Environmental Protection", "General Info", "", "deppressoffice@dep.nyc.gov", "718-595-6600"];
# press[press.length] = ["Food Policy", "Blackstone, Charles", "Chief of Staff", "cblackstone@foodpolicy.nyc.gov", "212-442-4482"];
# press[press.length] = ["Food Policy", "General Info", "", "nycfood@cityhall.nyc.gov", ""];
# press[press.length] = ["Finance", "Ryan Lavis", "Director of Public Information", "Lavisr@finance.nyc.gov", "212-602-7211"];
# press[press.length] = ["Fire Department", "", "", "fdnyopi@fdny.nyc.gov", "718-999-2056"];
# press[press.length] = ["Health + Hospitals", "Miller, Chris", "Senior Director for Media Relations", "millerc15@nychhc.org", "212-788-3339"];
# press[press.length] = ["Health and Mental Hygiene", "Gallague, Patrick", "Press Secretary", "pgallahue@health.nyc.gov", "347-396-4061"];
# press[press.length] = ["Health and Mental Hygiene", "General Info", "", "pressoffice@health.nyc.gov", "347-396-4177"];
# press[press.length] = ["Housing Authority", "Brancaccio, Barbara", "Chief Communications Officer", "Barbara.Brancaccio@nycha.nyc.gov", "212-306-6083"];
# press[press.length] = ["Housing Authority", "General Info", "", "Media@nycha.nyc.gov", ""];
# press[press.length] = ["Housing Development Corporation", "Ilana, Maier", "Press Secretary", "MaierI@hpd.nyc.gov", ""];
# press[press.length] = ["Housing Preservation & Development", "", "", "press@hpd.nyc.gov", ""];
# press[press.length] = ["Housing Recovery Operations", "Contact the Mayor's Press Office", "", "pressoffice@cityhall.nyc.gov", "212-788-2958"];
# press[press.length] = ["Human Rights Commission", "A. Rios Lua, Jose", "Executive Director, Communications and Marketing", "JRiosLua@cchr.nyc.gov", "212-416-0105"];
# press[press.length] = ["Immigrant Affairs", "Coronel, Shaina", "Press Secretary", "scoronel@cityhall.nyc.gov", "646-385-0334"];
# press[press.length] = ["International Affairs", "Contact the Mayor's Press Office", "", "pressoffice@cityhall.nyc.gov", "212-788-2958"];
# press[press.length] = ["Investigation", "Struzzi, Diane", "Director of Communications", "dstruzzi@doi.nyc.gov", "212-825-5931"];
# press[press.length] = ["Landmarks Preservation Commission", "Kersavage, Lisa", "Executive Director", "lkersavage@lpc.nyc.gov", "212-602-6385"];
# press[press.length] = ["Landmarks Preservation Commission", "", "", "lpcpressoffice@lpc.nyc.gov", ""];
# press[press.length] = ["Law Department", "Paolucci, Nick", "Director of Public Affairs & Press Secretary", "npaolucci@law.nyc.gov", "212-356-4010"];
# press[press.length] = ["Management & Budget", "Greenberg, Michael", "Communications Director", "GreenbergM@omb.nyc.gov", "212-788-5839"];
# press[press.length] = ["Mayor's Fund to Advance New York City", "Contact the Mayor's Press Office", "", "pressoffice@cityhall.nyc.gov", "212-788-2958"];
# press[press.length] = ["Mayor's Press Office", "", "", "pressoffice@cityhall.nyc.gov", "212-788-2958"];
# press[press.length] = ["Media and Entertainment", "Scharler, Melanie", "Associate Commissioner, Director of Strategic Communications", "Mscharler@media.nyc.gov", "212-974-4022"];
# press[press.length] = ["NYC 311", "Reda, Bill", "Director of Communications", "wreda@311.nyc.gov", "212-504-4292"];
# press[press.length] = ["Minority and Women Owned Business Enterprises", "Contact the Mayor's Press Office", "", "pressoffice@cityhall.nyc.gov", "212-788-2958"];
# press[press.length] = ["NYC & Company", "Townsend, Tiffany", "Executive Vice President, Global Communications", "ttownsend@nycgo.com", "732-682-6844"];
# press[press.length] = ["OATH (Administrative Trials and Hearings)", "Senigo, Marisa", "Deputy Commissioner for Public Affairs & Communications", "msenigo@oath.nyc.gov", "212-933-3080"];
# press[press.length] = ["Office of Technology and Innovation", "Legendre, Ray", "Senior Director of Communications", "rlegendre@oti.nyc.gov", "646-889-4106 "];
# press[press.length] = ["Office of Technology and Innovation", "General Info", "", "communications@doitt.nyc.gov", ""];
# press[press.length] = ["Parks & Recreation", "Press Office", "", "pressoffice@parks.nyc.gov", "212-360-1311"];
# press[press.length] = ["Police Department", "General Info", "", "DCPI@nypd.org", "646-610-6700"];
# press[press.length] = ["Probation", "Contact the Mayor's Press Office", "", "pressoffice@cityhall.nyc.gov", "212-788-2958"];
# press[press.length] = ["Public Engagement Unit", "", "Contact the Mayor's Press Office", "pressoffice@cityhall.nyc.gov", "212-788-2958"];
# press[press.length] = ["Sanitation", "Gragnani, Vincent", "Press Secretary", "vgragnani@dsny.nyc.gov", "646-885-5020"];
# press[press.length] = ["Sanitation", "General Info", "", "pressoffice@dsny.nyc.gov", "646-885-5020"];
# press[press.length] = ["Small Business Services", "Cho, Julianne", "Director of Communications", "jcho@sbs.nyc.gov", "212-618-8971"];
# press[press.length] = ["Social Services (DSS)", "Sharma, Neha", "Press Secretary", "sharman@dss.nyc.gov", "929-221-6632"];
# press[press.length] = ["Taxi & Limousine Commission", "Kersten, Jason", "Press Secretary", "Jason.Kersten@tlc.nyc.gov", "212-676-1070"];
# press[press.length] = ["Taxi & Limousine Commission", "Press and Media Contact", "", "press@tlc.nyc.gov", ""];
# press[press.length] = ["Transportation", "Barone, Vin", "First Deputy Press Secretary", "press@dot.nyc.gov", "212-839-6251"];
# press[press.length] = ["Transportation", "General Info", "", "press@dot.nyc.gov", "212-839-4850"];
# press[press.length] = ["Veterans Services", "Contact the Mayor's Press Office", "", "pressoffice@cityhall.nyc.gov", "212-788-2958"];
# press[press.length] = ["Youth & Community Development", "Zustovich, Mark", "Chief Public Information Offficer", "mzustovich@dycd.nyc.gov", "646-343-6740"];

# """

# # Parse the data
# df_press_contacts = parse_press_contacts(js_content)

# # Display the shape of the dataframe
# print(f"Shape of the dataframe: {df_press_contacts.shape}")

# # Display the first few rows of the dataframe
# print(df_press_contacts.head())

# # Save the dataframe to a CSV file
# #df_press_contacts.to_csv('agency_press_contacts.csv', index=False)
# #print("Data parsed and saved to CSV.")

Shape of the dataframe: (71, 7)
               AgencyName      PressContactName  \
0                 Actuary  Markoe-Boyd, Marlene   
1                   Aging                         
2               Buildings      Rudansky, Andrew   
3               Buildings                         
4  Chief Medical Examiner         Bolcer, Julie   

                                   PressContactTitle  \
0                         Director of Communications   
1                                                      
2                                    Press Secretary   
3                                                      
4  Executive Director of Public Affairs and Senio...   

              PressContactEmail PressContactPhone  \
0  mmarkoe-boyd@actuary.nyc.gov      212-312-0119   
1                                                   
2   ARudansky@buildings.nyc.gov      212-393-2072   
3                                                   
4          jbolcer@ocme.nyc.gov      212-447-2041   

     

In [ ]:
# !pip install fuzzywuzzy python-Levenshtein

# import pandas as pd
# from fuzzywuzzy import process
# import time

# # Wait for the installation to complete
# time.sleep(10)

In [ ]:
# df_press_contacts_combined.shape

Manually create crosswalk to standardized Name

In [ ]:
# import pandas as pd

# def create_crosswalk(df):
#     crosswalk = {}

#     for agency in df['AgencyName'].unique():
#         print(f"\nCurrent AgencyName: {agency}")
#         name = input("Enter the corresponding Name (or press Enter to skip): ")
#         if name:
#             crosswalk[agency] = name
#         else:
#             print(f"Skipped: {agency}")

#     return crosswalk

# # Assuming df_press_contacts is already loaded
# df_press_contacts_unique = df_press_contacts_combined.drop_duplicates(subset=['AgencyName'])

# # Create the crosswalk
# crosswalk = create_crosswalk(df_press_contacts_unique)

# # Add the Name field to df_press_contacts
# df_press_contacts['Name'] = df_press_contacts['AgencyName'].map(crosswalk)

# # Display the results
# print("\nCrosswalk created:")
# for agency, name in crosswalk.items():
#     print(f"{agency} -> {name}")

# print("\nUpdated df_press_contacts:")
# print(df_press_contacts[['AgencyName', 'Name']].head())

# # Create the press_contacts_crosswalk DataFrame
# press_contacts_crosswalk = pd.DataFrame(list(crosswalk.items()), columns=['AgencyName', 'Name'])

# # Save the press_contacts_crosswalk to a CSV file
# #press_contacts_crosswalk.to_csv('press_contacts_crosswalk.csv', index=False)
# #print("\nCrosswalk saved to 'press_contacts_crosswalk.csv'")

# # Save the updated df_press_contacts
# #df_press_contacts.to_csv('updated_press_contacts.csv', index=False)
# #print("Updated press contacts saved to 'updated_press_contacts.csv'")


Current AgencyName: Actuary
Enter the corresponding Name (or press Enter to skip): New York City Office of the Actuary

Current AgencyName: Aging
Enter the corresponding Name (or press Enter to skip): Department for the Aging

Current AgencyName: Buildings
Enter the corresponding Name (or press Enter to skip): Department of Buildings

Current AgencyName: Chief Medical Examiner
Enter the corresponding Name (or press Enter to skip): Office of the Chief Medical Examiner

Current AgencyName: Children's Services
Enter the corresponding Name (or press Enter to skip): Administration for Children's Services

Current AgencyName: City Planning
Enter the corresponding Name (or press Enter to skip): Department of City Planning

Current AgencyName: Citywide Administrative Services
Enter the corresponding Name (or press Enter to skip): Department of Citywide Administrative Services

Current AgencyName: Civilian Complaint Review Board
Enter the corresponding Name (or press Enter to skip): Civilian C

In [ ]:
# def combine_agency_info(group):
#     combined = pd.Series({
#         'AgencyName': group['AgencyName'].iloc[0],
#         'Name': group['Name'].iloc[0],
#         'PressContactName': [],
#         'PressContactTitle': [],
#         'PressContactEmail': [],
#         'PressContactPhone': [],
#         'GeneralInfoEmail': [],
#         'GeneralInfoPhone': []
#     })

#     for _, row in group.iterrows():
#         if pd.notna(row['GeneralInfoEmail']):
#             combined['GeneralInfoEmail'].append(row['GeneralInfoEmail'])
#         if pd.notna(row['GeneralInfoPhone']):
#             combined['GeneralInfoPhone'].append(row['GeneralInfoPhone'])
#         if pd.notna(row['PressContactName']):
#             combined['PressContactName'].append(row['PressContactName'])
#             combined['PressContactTitle'].append(row['PressContactTitle'])
#             combined['PressContactEmail'].append(row['PressContactEmail'])
#             combined['PressContactPhone'].append(row['PressContactPhone'])

#     # Remove duplicates and join lists into strings
#     for field in combined.index:
#         if isinstance(combined[field], list):
#             combined[field] = '; '.join(filter(None, dict.fromkeys(combined[field])))

#     return combined

# # Assuming df_press_contacts is already loaded
# df_press_contacts_combined = df_press_contacts.groupby('AgencyName').apply(combine_agency_info).reset_index(drop=True)

In [ ]:
# press_contacts_clean = df_press_contacts_combined

In [ ]:
# # Define the output file path
# output_file_path = '/content/drive/MyDrive/Projects/ODA/Agency Name Project/Output/press_contacts_clean.csv'

# # Export the dataframe to a CSV file
# press_contacts_clean.to_csv(output_file_path, index=False)

# print(f'Data exported successfully to {output_file_path}')

## Manually upload Press Contacts Clean file and merge it onto nyc_agencies_final

In [ ]:
# # File path
# file_path = '/content/drive/MyDrive/Projects/ODA/Agency Name Project/Output/press_contacts_clean.csv'

# # Read the CSV file
# press_contacts_clean = pd.read_csv(file_path)

In [ ]:
# # Assuming nyc_agencies_final and press_contacts_clean are already loaded

# # Merge the dataframes
# merged_df = pd.merge(nyc_agencies_final, press_contacts_clean, on='Name', how='left')

# # Rename the AgencyName column
# merged_df = merged_df.rename(columns={'AgencyName': 'Name - Agency Press Contact'})

# # Define the new column order
# new_column_order = (
#     nyc_agencies_final.columns.tolist()[:nyc_agencies_final.columns.get_loc('PrincipalOfficerTitle') + 1] +
#     ['PressContactName', 'PressContactTitle', 'PressContactEmail', 'PressContactPhone', 'GeneralInfoEmail', 'GeneralInfoPhone'] +
#     nyc_agencies_final.columns.tolist()[nyc_agencies_final.columns.get_loc('PrincipalOfficerTitle') + 1:nyc_agencies_final.columns.get_loc('Name - Checkbook') + 1] +
#     ['Name - Agency Press Contact'] +
#     nyc_agencies_final.columns.tolist()[nyc_agencies_final.columns.get_loc('Name - Checkbook') + 1:]
# )

# # Reorder the columns
# nyc_agencies_final = merged_df[new_column_order]

# Upload manualled QAed nyc_agencies_qa2 file

In [ ]:
# File path
file_path = '/content/drive/MyDrive/Projects/ODA/Agency Name Project/Output/nyc_agencies_qa2.csv'

# Read the CSV file
nyc_agencies_final = pd.read_csv(file_path)

In [ ]:
nyc_agencies_final.shape

(276, 42)

# Clean Up Tasks

Fix Budget Code Formatting

In [ ]:
import numpy as np

# Function to pad budget code with leading zeros
def pad_budget_code(code):
    if pd.isna(code):
        return np.nan
    try:
        # Convert to integer, then to 3-digit string
        return f"{int(code):03d}"
    except ValueError:
        # If conversion to int fails, return the original value
        return str(code)

# Apply the padding function to the BudgetCode column
nyc_agencies_final['BudgetCode'] = nyc_agencies_final['BudgetCode'].apply(pad_budget_code)

# Convert the column to string type, but keep NaN as NaN
nyc_agencies_final['BudgetCode'] = nyc_agencies_final['BudgetCode'].astype('object')

# Display a sample of the updated BudgetCode column
print(nyc_agencies_final[['Name', 'BudgetCode']].sample(10))

# Count of unique values in BudgetCode column
print("\nNumber of unique BudgetCode values:", nyc_agencies_final['BudgetCode'].nunique())

# Display rows where BudgetCode is not a 3-digit string or NaN
non_standard_codes = nyc_agencies_final[~nyc_agencies_final['BudgetCode'].str.match(r'^\d{3}$', na=True)]
if not non_standard_codes.empty:
    print("\nRows with non-standard BudgetCode values:")
    print(non_standard_codes[['Name', 'BudgetCode']])
else:
    print("\nAll BudgetCode values are now in the standard 3-digit format or NaN.")

                                                  Name BudgetCode
102            Hudson Yards Infrastructure Corporation        NaN
183                             NYC Health + Hospitals        819
196  Office of Building Energy and Emissions Perfor...        NaN
244             Sales Tax Asset Receivable Corporation        NaN
199                       Office of Climate Resiliency        NaN
232                     Office of the Utility Advocate        NaN
73   Deputy Chief Administrative Judge for the New ...        NaN
44                                    Community Boards        NaN
88               Financial Information Services Agency        NaN
33                         Civic Engagement Commission        NaN

Number of unique BudgetCode values: 57

All BudgetCode values are now in the standard 3-digit format or NaN.


In [ ]:
nyc_agencies_final.columns

Index(['NameID', 'Name', 'NameAlphabetized', 'OperationalStatus',
       'OrganizationType', 'Description', 'URL', 'ParentOrganization',
       'NYCReportingLine', 'AuthorizingAuthority', 'LegalCitation',
       'LegalCitationURL', 'LegalCitationText', 'LegalName',
       'CurrentOperatingName', 'AlternateNames', 'Acronym',
       'AlternateAcronyms', 'BudgetCode', 'PrincipalOfficerName',
       'PrincipalOfficerTitle', 'OpenDatasetsURL', 'Notes', 'Jurisdiction',
       'AdministrativeLevel', 'FoundingDate', 'SunsetDate',
       'SuccessorOrganization', 'URISlug', 'DateCreated', 'DateModified',
       'LastVerifiedDate', 'Name - NYC.gov Agency List',
       'Name - NYC.gov Mayor's Office', 'Name - NYC Open Data Portal',
       'Name - ODA', 'Name - CPO', 'Name - WeGov', 'Name - Greenbook',
       'Name - Checkbook', '_merge', '_PotentialDuplicate'],
      dtype='object')

Create NameWithAcronym and NameAlphabetizedWithAcronym fields

In [ ]:
# Create NameWithAcronym field
nyc_agencies_final['NameWithAcronym'] = nyc_agencies_final.apply(lambda row: f"{row['Name']} ({row['Acronym']})" if pd.notna(row['Acronym']) else row['Name'], axis=1)

# Create NameAlphabetizedWithAcronym field
nyc_agencies_final['NameAlphabetizedWithAcronym'] = nyc_agencies_final.apply(lambda row: f"{row['NameAlphabetized']} ({row['Acronym']})" if pd.notna(row['Acronym']) else row['NameAlphabetized'], axis=1)

In [ ]:
nyc_agencies_final.head()

,NameID,Name,NameAlphabetized,OperationalStatus,OrganizationType,Description,URL,ParentOrganization,NYCReportingLine,AuthorizingAuthority,...,Name - NYC Open Data Portal,Name - ODA,Name - CPO,Name - WeGov,Name - Greenbook,Name - Checkbook,_merge,_PotentialDuplicate,NameWithAcronym,NameAlphabetizedWithAcronym
0,administration for childrens services,Administration for Children's Services,"Children's Services, Administration for",Active,Mayoral Agency,The Administration for Children's Services (AC...,http://www.nyc.gov/acs,Mayor's Office,Deputy Mayor for Health and Human Services,New York City Charter,...,Administration for Children’s Services (ACS),Administration for Children's Services (ACS),Administration for Children's Services,Administration for Children's Services,"Children's Services, Administration for",Administration for Children's Services,both,False,Administration for Children's Services (ACS),"Children's Services, Administration for (ACS)"
1,advisory council to the department for the aging,Advisory Council to the Department for the Aging,"Aging, Advisory Council to the Department for the",Active,"Advisory, Oversight, or Regulatory Body",NaN,NaN,Department for the Aging,Department for the Aging,New York City Charter,...,NaN,NaN,NaN,NaN,NaN,NaN,right_only,False,Advisory Council to the Department for the Aging,"Aging, Advisory Council to the Department for the"
2,archival review board name,Archival Review Board,Archival Review Board,Active,Division,NaN,NaN,Department of Records and Information Systems,Department of Records and Information Systems,New York City Charter,...,NaN,NaN,NaN,NaN,NaN,NaN,right_only,False,Archival Review Board,Archival Review Board
3,art commission,Art Commission,Art Commission,Reorganized,"Advisory, Oversight, or Regulatory Body",NaN,NaN,NaN,NaN,New York City Charter,...,NaN,NaN,NaN,NaN,NaN,NaN,right_only,False,Art Commission,Art Commission
4,association for a better new york,Association for a Better New York,Association for a Better New York,Active,Non-Governmental Organization,NaN,https://abny.org/,NaN,NaN,New York State Law,...,NaN,Association for a Better New York (ABNY),NaN,NaN,NaN,NaN,both,False,Association for a Better New York (ABNY),Association for a Better New York (ABNY)


Split PrincipalOfficerName into PrincipalOfficerGivenName and PrincipalOfficerGivenName

In [ ]:
import pandas as pd

def split_name(name):
    if pd.isna(name):
        return pd.NA, pd.NA

    parts = name.split()

    if len(parts) == 2:
        return parts[0], parts[1]
    elif len(parts) == 3 and (len(parts[1]) == 1 or (len(parts[1]) == 2 and parts[1].endswith('.'))):
        return parts[0], parts[2]  # Ignore the middle initial
    elif len(parts) > 2:
        return None, None  # Flag for manual review
    else:
        return name, pd.NA  # For single-word names

def manual_name_split(row):
    print(f"\nFull Name: {row['PrincipalOfficerName']}")
    print("Please enter the Given Name and Family Name, ignoring any middle initials.")
    given_name = input("Enter Principal Officer Given Name: ")
    family_name = input("Enter Principal Officer Family Name: ")
    return given_name, family_name

# Create new columns without modifying the original DataFrame
given_names = []
family_names = []

for name in nyc_agencies_final['PrincipalOfficerName']:
    given, family = split_name(name)
    given_names.append(given)
    family_names.append(family)

# Add new columns to the DataFrame
nyc_agencies_final['PrincipalOfficerGivenName'] = given_names
nyc_agencies_final['PrincipalOfficerFamilyName'] = family_names

# Identify rows that need manual review
manual_review = nyc_agencies_final[
    (nyc_agencies_final['PrincipalOfficerGivenName'].isna()) &
    (nyc_agencies_final['PrincipalOfficerFamilyName'].isna()) &
    (nyc_agencies_final['PrincipalOfficerName'].notna())
]

# Perform manual review for flagged rows
for idx, row in manual_review.iterrows():
    given, family = manual_name_split(row)
    nyc_agencies_final.at[idx, 'PrincipalOfficerGivenName'] = given
    nyc_agencies_final.at[idx, 'PrincipalOfficerFamilyName'] = family

# Display the results
print(nyc_agencies_final[['PrincipalOfficerName', 'PrincipalOfficerGivenName', 'PrincipalOfficerFamilyName']].head())
print(f"\nTotal rows: {len(nyc_agencies_final)}")
print(f"Rows that required manual input: {len(manual_review)}")


Full Name: Félix V. Matos Rodríguez
Please enter the Given Name and Family Name, ignoring any middle initials.
Enter Principal Officer Given Name: Félix
Enter Principal Officer Family Name: Rodríguez

Full Name: Vilda Vera Mayuga
Please enter the Given Name and Family Name, ignoring any middle initials.
Enter Principal Officer Given Name: Vilda
Enter Principal Officer Family Name: Mayuga

Full Name: Dr. Ashwin Vasan
Please enter the Given Name and Family Name, ignoring any middle initials.
Enter Principal Officer Given Name: Ashwin
Enter Principal Officer Family Name: Vasan

Full Name: Adolfo Carrión Jr.
Please enter the Given Name and Family Name, ignoring any middle initials.
Enter Principal Officer Given Name: Adolfo
Enter Principal Officer Family Name: Carrión

Full Name: Molly Wasow Park
Please enter the Given Name and Family Name, ignoring any middle initials.
Enter Principal Officer Given Name: Molly
Enter Principal Officer Family Name: Park

Full Name: Dr. Mitchell Katz
Please

In [ ]:
nyc_agencies_final.shape

(276, 46)

In [ ]:
nyc_agencies_final.head()

,NameID,Name,NameAlphabetized,OperationalStatus,OrganizationType,Description,URL,ParentOrganization,NYCReportingLine,AuthorizingAuthority,...,Name - CPO,Name - WeGov,Name - Greenbook,Name - Checkbook,_merge,_PotentialDuplicate,NameWithAcronym,NameAlphabetizedWithAcronym,PrincipalOfficerGivenName,PrincipalOfficerFamilyName
0,administration for childrens services,Administration for Children's Services,"Children's Services, Administration for",Active,Mayoral Agency,The Administration for Children's Services (AC...,http://www.nyc.gov/acs,Mayor's Office,Deputy Mayor for Health and Human Services,New York City Charter,...,Administration for Children's Services,Administration for Children's Services,"Children's Services, Administration for",Administration for Children's Services,both,False,Administration for Children's Services (ACS),"Children's Services, Administration for (ACS)",Jess,Dannhauser
1,advisory council to the department for the aging,Advisory Council to the Department for the Aging,"Aging, Advisory Council to the Department for the",Active,"Advisory, Oversight, or Regulatory Body",NaN,NaN,Department for the Aging,Department for the Aging,New York City Charter,...,NaN,NaN,NaN,NaN,right_only,False,Advisory Council to the Department for the Aging,"Aging, Advisory Council to the Department for the",<NA>,<NA>
2,archival review board name,Archival Review Board,Archival Review Board,Active,Division,NaN,NaN,Department of Records and Information Systems,Department of Records and Information Systems,New York City Charter,...,NaN,NaN,NaN,NaN,right_only,False,Archival Review Board,Archival Review Board,<NA>,<NA>
3,art commission,Art Commission,Art Commission,Reorganized,"Advisory, Oversight, or Regulatory Body",NaN,NaN,NaN,NaN,New York City Charter,...,NaN,NaN,NaN,NaN,right_only,False,Art Commission,Art Commission,<NA>,<NA>
4,association for a better new york,Association for a Better New York,Association for a Better New York,Active,Non-Governmental Organization,NaN,https://abny.org/,NaN,NaN,New York State Law,...,NaN,NaN,NaN,NaN,both,False,Association for a Better New York (ABNY),Association for a Better New York (ABNY),<NA>,<NA>


In [ ]:
nyc_agencies_final.columns

Index(['NameID', 'Name', 'NameAlphabetized', 'OperationalStatus',
       'OrganizationType', 'Description', 'URL', 'ParentOrganization',
       'NYCReportingLine', 'AuthorizingAuthority', 'LegalCitation',
       'LegalCitationURL', 'LegalCitationText', 'LegalName',
       'CurrentOperatingName', 'AlternateNames', 'Acronym',
       'AlternateAcronyms', 'BudgetCode', 'PrincipalOfficerName',
       'PrincipalOfficerTitle', 'OpenDatasetsURL', 'Notes', 'Jurisdiction',
       'AdministrativeLevel', 'FoundingDate', 'SunsetDate',
       'SuccessorOrganization', 'URISlug', 'DateCreated', 'DateModified',
       'LastVerifiedDate', 'Name - NYC.gov Agency List',
       'Name - NYC.gov Mayor's Office', 'Name - NYC Open Data Portal',
       'Name - ODA', 'Name - CPO', 'Name - WeGov', 'Name - Greenbook',
       'Name - Checkbook', '_merge', '_PotentialDuplicate', 'NameWithAcronym',
       'NameAlphabetizedWithAcronym', 'PrincipalOfficerGivenName',
       'PrincipalOfficerFamilyName'],
      dtype='ob

In [ ]:
# Get unique values from the OperationalStatus field
unique_operational_status = nyc_agencies_final['OperationalStatus'].unique()

# Display the unique values
unique_operational_status

array(['Active', 'Reorganized', 'Out of Scope', 'Duplicate'], dtype=object)

In [ ]:
# Filter the DataFrame for records where OperationalStatus is 'Active' or 'Reorganized'
nyc_agencies_final = nyc_agencies_final[nyc_agencies_final['OperationalStatus'].isin(['Active', 'Reorganized'])]

# Filter the DataFrame for records where OperationalStatus is 'Active'
# nyc_agencies_final = nyc_agencies_final[nyc_agencies_final['OperationalStatus'] == 'Active']

In [ ]:
import unicodedata

def create_uri_slug(name):
    # Convert to lowercase
    slug = name.lower()

    # Remove accents
    slug = ''.join(c for c in unicodedata.normalize('NFD', slug)
                   if unicodedata.category(c) != 'Mn')

    # Replace spaces and special characters with hyphens
    slug = re.sub(r'[^\w\s-]', '', slug)
    slug = re.sub(r'[\s_-]+', '-', slug)

    # Remove leading/trailing hyphens
    slug = slug.strip('-')

    return slug

# Apply the function to create URISlug
nyc_agencies_final['URISlug'] = nyc_agencies_final['Name'].apply(create_uri_slug)

# Display a sample of the results
print(nyc_agencies_final[['Name', 'URISlug']].head(10))

# Check for any duplicate slugs
duplicate_slugs = nyc_agencies_final[nyc_agencies_final.duplicated(subset='URISlug', keep=False)]
if not duplicate_slugs.empty:
    print("\nWarning: Duplicate slugs found:")
    print(duplicate_slugs[['Name', 'URISlug']])
else:
    print("\nNo duplicate slugs found.")

# Count of non-empty URISlug values
non_empty_count = nyc_agencies_final['URISlug'].notna().sum()
print(f"\nNumber of non-empty URISlug values: {non_empty_count}")

# Save the updated DataFrame if needed
# nyc_agencies_final.to_csv('nyc_agencies_final_with_slugs.csv', index=False)

                                               Name  \
0            Administration for Children's Services   
1  Advisory Council to the Department for the Aging   
2                             Archival Review Board   
3                                    Art Commission   
4                 Association for a Better New York   
5                                   Audit Committee   
6                               Board of Correction   
7              Board of Education Retirement System   
8                                Board of Elections   
9                                   Board of Health   

                                            URISlug  
0             administration-for-childrens-services  
1  advisory-council-to-the-department-for-the-aging  
2                             archival-review-board  
3                                    art-commission  
4                 association-for-a-better-new-york  
5                                   audit-committee  
6               

#QA

In [ ]:
!pip install pyspellchecker

In [ ]:
import pandas as pd
import re
from spellchecker import SpellChecker
from datetime import datetime
import os

# Load your dataset
df = nyc_agencies_final

def check_special_characters(df, columns_to_check):
    special_char_issues = []
    pattern = re.compile(r'[^a-zA-Z0-9\s]')
    for col in columns_to_check:
        issues = df[df[col].apply(lambda x: bool(pattern.search(str(x))) if pd.notna(x) else False)]
        if not issues.empty:
            special_char_issues.append(f"Special characters found in {col}:\n{issues[['Name', col]]}\n")
    return special_char_issues

def check_spelling(df, columns_to_check):
    spell = SpellChecker()
    spelling_issues = []
    for col in columns_to_check:
        for idx, value in df[col].items():
            if pd.isna(value):  # Skip NaN values
                continue
            if not isinstance(value, str):  # Convert non-string values to string
                value = str(value)
            words = value.split()
            misspelled = spell.unknown(words)
            if misspelled:
                spelling_issues.append(f"Potential misspellings in {col} for {df.loc[idx, 'Name']}: {misspelled}\n")
    return spelling_issues

def check_file_format(df):
    format_issues = []

    # Check for required columns
    required_columns = ['Name', 'OperationalStatus', 'OrganizationType', 'Description', 'URL']  # Add all required columns
    missing_columns = set(required_columns) - set(df.columns)
    if missing_columns:
        format_issues.append(f"Missing required columns: {missing_columns}\n")

    # Check for empty columns
    empty_columns = df.columns[df.isna().all()].tolist()
    if empty_columns:
        format_issues.append(f"Empty columns: {empty_columns}\n")

    # Check data types
    expected_types = {
        'Name': 'object',
        'OperationalStatus': 'object',
        'OrganizationType': 'object',
        'BudgetCode': 'object',
        'FoundingDate': 'object',  # Assuming date is stored as string
        # Add more columns and their expected types
    }
    for col, expected_type in expected_types.items():
        if col in df.columns and df[col].dtype != expected_type:
            format_issues.append(f"Incorrect data type for {col}. Expected {expected_type}, got {df[col].dtype}\n")

    # Check date format
    date_columns = ['FoundingDate', 'SunsetDate', 'DateCreated', 'DateModified', 'LastVerifiedDate']
    for col in date_columns:
        if col in df.columns:
            incorrect_dates = df[~df[col].apply(lambda x: bool(re.match(r'\d{4}-\d{2}-\d{2}', str(x))) if pd.notna(x) else True)]
            if not incorrect_dates.empty:
                format_issues.append(f"Incorrect date format in {col}:\n{incorrect_dates[['Name', col]]}\n")

    return format_issues

def check_data_consistency(df):
    consistency_issues = []

    # Check for duplicates
    duplicates = df[df.duplicated(subset=['Name'], keep=False)]
    if not duplicates.empty:
        consistency_issues.append(f"Duplicate entries found:\n{duplicates[['Name']]}\n")

    # Check if Acronym is consistent with Name
    def check_acronym(row):
        if pd.isna(row['Acronym']) or pd.isna(row['Name']):
            return False
        return str(row['Acronym']) not in str(row['Name'])

    inconsistent_acronyms = df[df.apply(check_acronym, axis=1)]
    if not inconsistent_acronyms.empty:
        consistency_issues.append(f"Inconsistent Acronyms:\n{inconsistent_acronyms[['Name', 'Acronym']]}\n")

    return consistency_issues

# Run QA checks
try:
    special_char_issues = check_special_characters(df, ['Name', 'Description', 'URL'])
    spelling_issues = check_spelling(df, ['Name', 'Description'])
    format_issues = check_file_format(df)
    consistency_issues = check_data_consistency(df)

    # Compile QA report
    qa_report = "QA Report\n\n"
    qa_report += "Special Character Issues:\n" + "".join(special_char_issues) + "\n"
    qa_report += "Spelling Issues:\n" + "".join(spelling_issues) + "\n"
    qa_report += "File Format Issues:\n" + "".join(format_issues) + "\n"
    qa_report += "Data Consistency Issues:\n" + "".join(consistency_issues)

    # Save QA report
    output_file_path = '/content/drive/MyDrive/Projects/ODA/Agency Name Project/Output/'
    os.makedirs(output_file_path, exist_ok=True)
    qa_report_path = os.path.join(output_file_path, 'qa_report.txt')
    with open(qa_report_path, 'w') as f:
        f.write(qa_report)

    print(f"QA complete. Report saved to '{qa_report_path}'")

except Exception as e:
    print(f"An error occurred during the QA process: {str(e)}")
    print("Please check your data and the QA functions.")

QA complete. Report saved to '/content/drive/MyDrive/Projects/ODA/Agency Name Project/Output/qa_report.txt'


# Rename and fill date fields

In [ ]:
# Rename FoundingDate to FoundingYear and SunsetDate to SunsetYear
nyc_agencies_final = nyc_agencies_final.rename(columns={
    'FoundingDate': 'FoundingYear',
    'SunsetDate': 'SunsetYear'
})

# Convert FoundingYear and SunsetYear to integer years
nyc_agencies_final['FoundingYear'] = pd.to_numeric(nyc_agencies_final['FoundingYear'], errors='coerce').astype('Int64')
nyc_agencies_final['SunsetYear'] = pd.to_numeric(nyc_agencies_final['SunsetYear'], errors='coerce').astype('Int64')

In [ ]:
# Fill DateCreated with today's date (7-29-24)
today_date = datetime(2024, 7, 29).date()
nyc_agencies_final['DateCreated'] = today_date

In [ ]:
# Rename the 'OrganizationType' column to 'PreliminaryOrganizationType'
nyc_agencies_final = nyc_agencies_final.rename(columns={'OrganizationType': 'PreliminaryOrganizationType'})

In [ ]:
nyc_agencies_final.columns

Index(['NameID', 'Name', 'NameAlphabetized', 'OperationalStatus',
       'PreliminaryOrganizationType', 'Description', 'URL',
       'ParentOrganization', 'NYCReportingLine', 'AuthorizingAuthority',
       'LegalCitation', 'LegalCitationURL', 'LegalCitationText', 'LegalName',
       'CurrentOperatingName', 'AlternateNames', 'Acronym',
       'AlternateAcronyms', 'BudgetCode', 'PrincipalOfficerName',
       'PrincipalOfficerTitle', 'OpenDatasetsURL', 'Notes', 'Jurisdiction',
       'AdministrativeLevel', 'FoundingYear', 'SunsetYear',
       'SuccessorOrganization', 'URISlug', 'DateCreated', 'DateModified',
       'LastVerifiedDate', 'Name - NYC.gov Agency List',
       'Name - NYC.gov Mayor's Office', 'Name - NYC Open Data Portal',
       'Name - ODA', 'Name - CPO', 'Name - WeGov', 'Name - Greenbook',
       'Name - Checkbook', '_merge', '_PotentialDuplicate', 'NameWithAcronym',
       'NameAlphabetizedWithAcronym', 'PrincipalOfficerGivenName',
       'PrincipalOfficerFamilyName'],
    

# Final Export

In [ ]:
# List of columns to drop
columns_to_drop = ['NameID', 'CurrentOperatingName', 'Jurisdiction', 'AdministrativeLevel', 'SuccessorOrganization', '_merge', '_PotentialDuplicate']

# Create new DataFrame without the specified columns
nyc_agencies_export = nyc_agencies_final.drop(columns=columns_to_drop)

In [ ]:
# Define the output file path
output_file_path = '/content/drive/MyDrive/Projects/ODA/Agency Name Project/Output/nyc_agencies_export.csv'

# Export the dataframe to a CSV file
nyc_agencies_export.to_csv(output_file_path, index=False)

print(f'Data exported successfully to {output_file_path}')

Data exported successfully to /content/drive/MyDrive/Projects/ODA/Agency Name Project/Output/nyc_agencies_export.csv
